# Framework Investigativo Completo: Ruído Quântico Benéfico em VQCs\n\n[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarceloClaro/Beneficial-Quantum-Noise-in-Variational-Quantum-Classifiers/blob/main/notebooks/03_reproducao_experimentos.ipynb)\n\n---\n\n## 📋 Visão Geral\n\nEste notebook implementa **integralmente** o Framework Investigativo v7.2 do artigo científico \n*"From Obstacle to Opportunity: Harnessing Beneficial Quantum Noise in Variational Classifiers"*,\nmantendo rigor científico QUALIS A1.\n\n### 🎯 Objetivos\n\n1. **Reproduzir todas as funções** do arquivo `framework_investigativo_completo.py`\n2. **Demonstrar regime de ruído quântico benéfico** com rigor estatístico\n3. **Manter padrões QUALIS A1**: reprodutibilidade, análise estatística rigorosa\n4. **Dupla perspectiva**: acessível para iniciantes, rigorosa para especialistas\n\n### 👥 Público-Alvo\n\n#### 👶 Iniciantes\n- Conceitos básicos explicados com analogias\n- Visualizações intuitivas\n- Passo a passo detalhado\n\n#### 🎓 Especialistas\n- Rigor matemático completo (Lindblad, von Neumann)\n- Análises estatísticas avançadas (ANOVA, Cohen's d, post-hoc)\n- Referências científicas\n- Compatibilidade com hardware real\n\n### 📚 Referências Fundamentais\n\n- **Nielsen & Chuang (2010)**: *Quantum Computation and Quantum Information*\n- **Preskill (2018)**: *Quantum Computing in the NISQ era*\n- **Cerezo et al. (2021)**: *Variational quantum algorithms*, Nature Reviews Physics\n- **Benedetti et al. (2019)**: *Parameterized quantum circuits as ML models*\n\n---

## 1. Configuração e Instalação\n\n### 💡 Para Iniciantes\nExecute a célula abaixo para instalar todas as dependências necessárias.\n\n### 🎓 Para Especialistas\nDependências com versões específicas para reprodutibilidade QUALIS A1.

In [ ]:
%%capture\n# Instalação de dependências (modo silencioso)\n!pip install pennylane numpy pandas scikit-learn scipy statsmodels plotly optuna\n\nprint('✓ Dependências instaladas com sucesso!')

## 2. Imports Centralizados\n\n### 💡 Para Iniciantes\nImportando todas as bibliotecas necessárias.\n\n### 🎓 Para Especialistas\nOrganização segundo PEP 8, imports agrupados logicamente.

In [ ]:
# Imports centralizados no topo do arquivo (PEP 8)
import os
import json
import time
import logging
from pathlib import Path
from datetime import datetime
from typing import TYPE_CHECKING, Any, Dict, Optional

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import datasets as sk_datasets
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# Estatística
from scipy.stats import f_oneway, ttest_ind
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Visualização
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Otimização Bayesiana (opcional)
try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import MedianPruner
    OPTUNA_AVAILABLE = True
except ImportError:
    OPTUNA_AVAILABLE = False

# Inicializar logging com formato QUALIS A1 (rigor científico)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(name)-20s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

# Configurar\n\nprint('✓ Imports realizados com sucesso!')

## 3. Constantes Fundamentais da Física Quântica\n\n### 💡 Para Iniciantes\nValores numéricos fundamentais usados em computação quântica.\n\n### 🎓 Para Especialistas\nConstantes baseadas em CODATA 2018 e valores aceitos pela comunidade científica.\nImplementação rigorosa das constantes fundamentais de Planck, Boltzmann, etc.

In [ ]:
class ConstantesFundamentais:
    """
    Constantes matemáticas e físicas para inicialização de parâmetros.

    Referências:
    - Constantes matemáticas: Weisstein, "MathWorld"
    - Constantes quânticas: CODATA 2018 (Mohr et al., 2019)
    - Normalização: Grant et al. (2019). Quantum.
    """

    # Constantes Matemáticas
    PI = np.pi                          # π ≈ 3.14159
    E = np.e                            # e ≈ 2.71828
    PHI = (1 + np.sqrt(5)) / 2         # φ ≈ 1.61803 (Razão Áurea)
    SQRT2 = np.sqrt(2)                  # √2 ≈ 1.41421
    LN2 = np.log(2)                     # ln(2) ≈ 0.69315
    GAMMA = 0.5772156649                # γ (Euler-Mascheroni)

    # Constantes Quânticas (CODATA 2018)
    HBAR = 1.054571817e-34              # ℏ (constante de Planck reduzida) [J·s]
    ALPHA = 7.2973525693e-3             # α (constante de estrutura fina) [adimensional]
    RYDBERG = 10973731.568160           # R∞ (constante de Rydberg) [m⁻¹]

    @classmethod
    def normalizar(cls, valores):
        """
        Normaliza valores para [-π, π] usando escala logarítmica.

        Referência: Grant et al. (2019). "An initialization strategy for
        addressing barren plateaus in parametrized quantum circuits." Quantum.

        Motivação: Constantes fundamentais abrangem 40 ordens de magnitude.
        Escala logarítmica mapeia para intervalo adequado para portas de rotação.
        """
        log_vals = np.log10(np.abs(valores) + 1e-10)
        norm = (log_vals - log_vals.min()) / (log_vals.max() - log_vals.min() + 1e-10)
        return -np.pi + norm * 2 * np.pi

    @classmethod
    def inicializar(cls, n_params, estrategia='aleatorio', seed=42):
        """
        Inicializa parâmetros com diferentes estratégias.

        Args:
            n_params: Número de parâmetros
            estrategia: 'matematico', 'quantico', ou 'aleatorio'
            seed: Semente aleatória para reprodutibilidade

        Returns:
            Array PennyLane com requires_grad=True
        """
        np.random.seed(seed)

        if estrategia == 'matematico':
            # Usa constantes matemáticas fundamentais
            const = np.array([cls.PI, cls.E, cls.PHI, cls.SQRT2, cls.LN2, cls.GAMMA])
            n_rep = int(np.ceil(n_params / len(const)))
            params = np.tile(const, n_rep)[:n_params]
            # Adiciona ruído gaussiano pequeno para quebrar simetria
            params += np.random.normal(0, 0.1, n_params)
            return pnp.array(cls.normalizar(params), requires_grad=True)

        elif estrategia == 'quantico':
            # Usa constantes físicas quânticas (CODATA 2018)
            const = np.array([cls.HBAR, cls.ALPHA, cls.RYDBERG])
            n_rep = int(np.ceil(n_params / len(const)))
            params = np.tile(const, n_rep)[:n_params]
            params += np.random.normal(0, 0.1, n_params)
            return pnp.array(cls.normalizar(params), requires_grad=True)

        elif estrategia == 'fibonacci_spiral':\n\nprint('✓ Classe ConstantesFundamentais definida!')

## 4. Modelos de Ruído Quântico\n\n### 💡 Para Iniciantes\nRuído quântico é como "estática" que afeta qubits. Diferentes tipos de ruído\nsimulam imperfeições reais do hardware quântico.\n\n### 🎓 Para Especialistas\nImplementação via **operadores de Kraus** e **Master Equation de Lindblad**:\n\n$$\\frac{d\\rho}{dt} = -i[H, \\rho] + \\sum_k \\left( L_k \\rho L_k^\\dagger - \\frac{1}{2}\\{L_k^\\dagger L_k, \\rho\\} \\right)$$\n\nModelos implementados:\n- **Depolarizante**: canal mais geral, mistura com estado maximamente misto\n- **Amplitude Damping**: perda de energia (relaxação T1)\n- **Phase Damping**: perda de coerência de fase (T2)\n- **Bit Flip, Phase Flip**: erros discretos\n- **Thermal, Pink Noise, Readout Error**: modelos avançados

In [ ]:
class ModeloRuido:
    """Classe base para modelos de ruído quântico."""
    def __init__(self, nivel=0.01):
        self.nivel = nivel
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply quantum noise to circuit qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
            
        Raises:
            NotImplementedError: Must be implemented by subclasses
        """
        raise NotImplementedError

class RuidoThermal(ModeloRuido):
    """Thermal Relaxation Error: aproxima T1/T2 com canais de amplitude e fase."""
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply thermal relaxation noise to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        p = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.AmplitudeDamping(p, wires=i)
            qml.PhaseDamping(p, wires=i)

class RuidoBitFlip(ModeloRuido):
    """Bit-Flip Error (X)."""
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply bit-flip noise (X gate with probability p) to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        p = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.BitFlip(p, wires=i)

class RuidoPhaseFlip(ModeloRuido):
    """Phase-Flip Error (Z)."""
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply phase-flip noise (Z gate with probability p) to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        p = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.PhaseFlip(p, wires=i)

class RuidoPinkNoise(ModeloRuido):
    """1/f Noise (Pink): usa PhaseDamping com variação por qubit."""
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply 1/f (pink) noise using phase damping with per-qubit variation.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
            
        Notes:
            Simulates low-frequency noise with Gaussian-distributed intensity per qubit
        """
        p = self.nivel if nivel_override is None else nivel_override
        pink = np.abs(np.random.normal(loc=0, scale=p, size=n_qubits))
        for i in range(n_qubits):
            qml.PhaseDamping(float(min(1.0, pink[i])) , wires=i)

class RuidoReadoutError(ModeloRuido):
    """Readout Error (aproximação via BitFlip após operações)."""
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply readout error approximated via bit-flip operations.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
            
        Notes:
            Models measurement errors in quantum hardware
        """
        p = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.BitFlip(p, wires=i)


class RuidoDepolarizante(ModeloRuido):
    """
    Ruído de Depolarização: ρ → (1-p)ρ + p·I/2

    Referência: Preskill (2018). "Quantum Computing in the NISQ era." Quantum.

    Descrição: Erro genérico mais comum em qubits supercondutores (IBM, Google).
    O estado quântico é substituído pelo estado maximamente misto com probabilidade p.

    Operadores de Kraus:
    K₀ = √(1-p) I
    K₁ = √(p/3) X
    K₂ = √(p/3) Y
    K₃ = √(p/3) Z
    """
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply depolarizing channel to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        p = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.DepolarizingChannel(p, wires=i)


class RuidoAmplitudeDamping(ModeloRuido):
    """
    Amplitude Damping: Relaxamento T1 (|1⟩ → |0⟩)

    Referência: Clerk et al. (2010). "Introduction to quantum noise." Rev. Mod. Phys.

    Descrição: Perda de energia do qubit para o ambiente. Modela decaimento
    exponencial com tempo característico T1. Comum em qubits supercondutores.

    Operadores de Kraus:
    K₀ = [[1, 0], [0, √(1-γ)]]
    K₁ = [[0, √γ], [0, 0]]
    """
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply amplitude damping channel to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        g = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.AmplitudeDamping(g, wires=i)


class RuidoPhaseDamping(ModeloRuido):
    """
    Phase Damping: Decoerência T2 (perda de fase)

    Referência: Schlosshauer (2007). "Decoherence and the Quantum-to-Classical Transition"

    Descrição: Perda de informação de fase sem perda de energia. Modela
    decoerência com tempo característico T2. Importante em qubits de spin.

    Operadores de Kraus:
    K₀ = [[1, 0], [0, √(1-λ)]]
    K₁ = [[0, 0], [0, √λ]]
    """
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply phase damping channel to all qubits.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        lmb = self.nivel if nivel_override is None else nivel_override
        for i in range(n_qubits):
            qml.PhaseDamping(lmb, wires=i)


# ===================== NOVOS MODELOS DE RUÍDO =====================
class RuidoCrosstalk(ModeloRuido):
    """
    Ruído de Cross-Talk: Erros correlacionados entre qubits vizinhos

    Referência: Kandala et al. (2019). "Error mitigation extends the computational reach of a noisy quantum processor." Nature.

    Descrição: Aplica um canal de ruído correlacionado entre pares de qubits vizinhos (ex: CNOT com ruído).
    """
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply correlated crosstalk noise between neighboring qubit pairs.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        p = self.nivel if nivel_override is None else nivel_override
        # Aplica canal de ruído correlacionado entre pares vizinhos
        for i in range(n_qubits):
            # Canal de ruído correlacionado: DepolarizingChannel em ambos os qubits simultaneamente
            qml.DepolarizingChannel(p, wires=i)
            qml.DepolarizingChannel(p, wires=(i+1)%n_qubits)
            # Cross-talk: canal extra entre pares
            qml.CNOT(wires=[i, (i+1)%n_qubits])
            qml.DepolarizingChannel(p, wires=(i+1)%n_qubits)
            qml.CNOT(wires=[i, (i+1)%n_qubits])


class RuidoCorrelacionado(ModeloRuido):
    """
    Ruído Correlacionado Global: Erros coletivos afetando todos os qubits

    Referência: Greenbaum (2015). "Introduction to Quantum Gate Set Tomography."

    Descrição: Aplica um canal de ruído coletivo (ex: PhaseDamping global) a todos os qubits simultaneamente.
    """
    def aplicar(self, n_qubits, nivel_override=None):
        """
        Apply global correlated noise affecting all qubits collectively.
        
        Args:
            n_qubits: Number of qubits in the circuit
            nivel_override: Optional noise level override (uses self.nivel if None)
        """
        lmb = self.nivel if nivel_override is None else nivel_override
        # Aplica PhaseDamping global a todos os qubits (efeito coletivo)
        for i in range(n_qubits):
            qml.PhaseDamping(lmb, wires=i)
        # Canal coletivo: aplica uma operação global (exemplo: ruído de fase global)
        # PennyLane não tem canal global nativo, mas pode-se simular aplicando em todos simultaneamente
        # Alternativamente, pode-se aplicar um canal customizado aqui se necessário


# Dicionário de modelos disponíveis
MODELOS_RUIDO = {
    'sem_ruido': None,
    'depolarizante': RuidoDepolarizante,
    'amplitude_damping': RuidoAmplitudeDamping,
    'phase_damping': RuidoPhaseDamping,
    'crosstalk': RuidoCrosstalk,
    'thermal': RuidoThermal,
    'correlated_noise': RuidoCorrelacionado,
    'bit_flip': RuidoBitFlip,
    'phase_flip': RuidoPhaseFlip,
    'pink_noise': RuidoPinkNoise,
    'readout_error': RuidoReadoutError
}


# ============================================================================
# MÓDULO 3: ARQUITETURAS DE CIRCUITOS QUÂNTICOS\n\nprint('✓ Modelos de ruído definidos!')

## 5. Arquiteturas de Circuitos Quânticos Variacionais\n\n### 💡 Para Iniciantes\nCircuitos quânticos são como "programas" que rodam em computadores quânticos.\nDiferentes arquiteturas testam diferentes maneiras de processar dados.\n\n### 🎓 Para Especialistas\nImplementamos 9 arquiteturas variacionais:\n1. **Hardware Efficient**: otimizado para topologia de hardware real\n2. **Strongly Entangling**: máximo emaranhamento entre qubits\n3. **Tree**: estrutura em árvore para reduzir porta CNOT\n4. **QAOA-like**: inspirado em Quantum Approximate Optimization\n5. **Alternating Layers**: alternância RX-RY-RZ com CNOTs\n6. **Star Entanglement**: qubit central conectado a todos\n7. **Brickwork**: padrão de tijolos alternados\n8. **Random Entangling**: emaranhamento estocástico\n9. **Básico**: arquitetura simples de referência

In [ ]:
def circuito_hardware_efficient(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Hardware-Efficient Ansatz: RY + CNOT em camadas alternadas
    """
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)
        for i in range(0, n_qubits-1, 2):
            qml.CNOT(wires=[i, i+1])
        for i in range(1, n_qubits-1, 2):
            qml.CNOT(wires=[i, i+1])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_tree(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Tree Entanglement: Emaranhamento em árvore binária
    """
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)
        step = 1
        while step < n_qubits:
            for i in range(0, n_qubits-step, step*2):
                qml.CNOT(wires=[i, i+step])
            step *= 2
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_qaoa(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    QAOA-like Ansatz: Alterna RX, RZZ, RX
    """
    for i in range(min(len(x), n_qubits)):
        qml.RX(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RX(weights[camada * n_qubits + i], wires=i)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
            qml.RZ(weights[n_camadas * n_qubits + camada * (n_qubits-1) + i], wires=i+1)
            qml.CNOT(wires=[i, i+1])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_alternating_layers(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Alternating Layers: RX, RY, CNOT em padrão alternado
    """
    for i in range(min(len(x), n_qubits)):
        qml.RX(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RX(weights[camada * n_qubits + i], wires=i)
            qml.RY(weights[n_camadas * n_qubits + camada * n_qubits + i], wires=i)
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_star_entanglement(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Star Entanglement: Qubit 0 central, CNOT(0, i)
    """
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)
        for i in range(1, n_qubits):
            qml.CNOT(wires=[0, i])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_brickwork(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Brickwork: padrão de CNOTs em "tijolos"
    """
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)
        for i in range(0, n_qubits-1, 2):
            qml.CNOT(wires=[i, i+1])
        for i in range(1, n_qubits-1, 2):
            qml.CNOT(wires=[i, i+1])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))

def circuito_random_entangling(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Random Entangling: CNOTs entre pares aleatórios por camada
    """
    import random
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)
    for camada in range(n_camadas):
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)
        pares = [(i, j) for i in range(n_qubits) for j in range(i+1, n_qubits)]
        random.shuffle(pares)
        for i, j in pares[:n_qubits//2]:
            qml.CNOT(wires=[i, j])
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)
    return qml.expval(qml.PauliZ(0))
# ============================================================================

def circuito_basico(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Circuito Básico: RY encoding + RY rotations + CNOT ring

    Referência: Farhi & Neven (2018). "Classification with quantum neural networks
    on near term processors." arXiv:1802.06002

    Estrutura:
    1. Encoding: RY(π·x[i]) em cada qubit
    2. Camadas variacionais (repetidas L vezes):
       - RY(θ[i]) em cada qubit
       - CNOT em anel: CNOT(i, i+1 mod n)
    3. Medição: ⟨Z₀⟩

    Complexidade: O(n_qubits × n_camadas)
    Parâmetros: n_qubits × n_camadas

    Vantagens:
    - Simples e rápido
    - Bom para prototipagem
    - Baixo número de parâmetros
    """
    # 1. Encoding de dados
    for i in range(min(len(x), n_qubits)):
        qml.RY(np.pi * x[i], wires=i)

    # 2. Camadas variacionais
    for camada in range(n_camadas):
        # Rotações parametrizadas
        for i in range(n_qubits):
            qml.RY(weights[camada * n_qubits + i], wires=i)

        # Emaranhamento (CNOT em anel)
        for i in range(n_qubits):
            qml.CNOT(wires=[i, (i + 1) % n_qubits])

        # Aplicar ruído após cada camada (se especificado)
        if modelo_ruido:
            modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)

    # 3. Medição
    return qml.expval(qml.PauliZ(0))


def circuito_strongly_entangling(weights, x, n_qubits, n_camadas, modelo_ruido=None, nivel_ruido_runtime=None):
    """
    Strongly Entangling Layers (PennyLane template)

    Referência: Schuld et al. (2020). "Circuit-centric quantum classifiers."
    Physical Review A, 101(3), 032308.

    Estrutura:
    1. Encoding: AngleEmbedding (RY em cada qubit)
    2. StronglyEntanglingLayers (template PennyLane):
       - Rot(θ, φ, ω) em cada qubit (3 rotações arbitrárias)
       - CNOT(i, j) para todos i < j (emaranhamento completo)
    3. Medição: ⟨Z₀⟩

    Complexidade: O(n_qubits² × n_camadas)
    Parâmetros: n_qubits × n_camadas × 3

    Vantagens:
    - Alta capacidade expressiva
    - Emaranhamento forte
    - Template otimizado do PennyLane
    """
    # 1. Encoding de dados
    qml.AngleEmbedding(x, wires=range(n_qubits), rotation='Y')

    # 2. Camadas fortemente emaranhadas
    weights_reshaped = weights.reshape(n_camadas, n_qubits, 3)
    qml.StronglyEntanglingLayers(weights_reshaped, wires=range(n_qubits))

    # 3. Aplicar ruído (se especificado)
    if modelo_ruido:
        modelo_ruido.aplicar(n_qubits, nivel_override=nivel_ruido_runtime)

    # 4. Medição
    return qml.expval(qml.PauliZ(0))


# Dicionário de arquiteturas disponíveis
ARQUITETURAS = {
    'basic_entangler': (circuito_basico, lambda nq, nc: nc * nq),
    'strongly_entangling': (circuito_strongly_entangling, lambda nq, nc: nc * nq * 3),
    'hardware_efficient': (circuito_hardware_efficient, lambda nq, nc: nc * nq),
    'tree': (circuito_tree, lambda nq, nc: nc * nq),
    'qaoa': (circuito_qaoa, lambda nq, nc: nc * nq + nc * (nq-1)),
    'alternating_layers': (circuito_alternating_layers, lambda nq, nc: 2 * nc * nq),
    'star_entanglement': (circuito_star_entanglement, lambda nq, nc: nc * nq),
    'brickwork': (circuito_brickwork, lambda nq, nc: nc * nq),
    'random_entangling': (circuito_random_entangling, lambda nq, nc: nc * nq)
}


# ============================================================================
# MÓDULO 4: CLASSIFICADOR QUÂNTICO VARIACIONAL (VQC)
# ============================================================================\n\nprint('✓ Arquiteturas de circuitos definidas!')

## 6. Classificador Quântico Variacional (VQC)\n\n### 💡 Para Iniciantes\nO VQC é como uma rede neural quântica que aprende a classificar dados.\nEle ajusta parâmetros internos para melhorar suas previsões.\n\n### 🎓 Para Especialistas\nImplementação compatível com scikit-learn (BaseEstimator, ClassifierMixin).\n\n**Método de otimização**: Gradient Descent com Parameter Shift Rule\n$$\\frac{\\partial}{\\partial \\theta_i} \\langle \\psi(\\theta) | H | \\psi(\\theta) \\rangle = \\frac{1}{2}\\left[ \\langle \\psi(\\theta + \\pi/2 e_i) | H | \\psi(\\theta + \\pi/2 e_i) \\rangle - \\langle \\psi(\\theta - \\pi/2 e_i) | H | \\psi(\\theta - \\pi/2 e_i) \\rangle \\right]$$\n\nFuncionalidades:\n- Múltiplas funções de custo (MSE, Cross-Entropy, Hinge)\n- Detecção de Barren Plateaus\n- Monitoramento de emaranhamento\n- Schedule adaptativo de ruído

In [ ]:
class ClassificadorVQC(BaseEstimator, ClassifierMixin):
    """
    Classificador Quântico Variacional.

    Referências:
    - Schuld et al. (2020). "Circuit-centric quantum classifiers." Phys. Rev. A.
    - Mitarai et al. (2018). "Quantum circuit learning." Phys. Rev. A.
    - Bergholm et al. (2018). "PennyLane: Automatic differentiation." arXiv:1811.04968

    Implementa interface scikit-learn (BaseEstimator, ClassifierMixin) para
    compatibilidade com pipelines de ML clássico.
    """

    def __init__(self, n_qubits=4, n_camadas=2, arquitetura='basico',
                 estrategia_init='aleatorio', tipo_ruido='sem_ruido', nivel_ruido=0.01,
                 taxa_aprendizado=0.01, n_epocas=20, batch_size=32, seed=42,
                 ruido_schedule=None, ruido_inicial=None, ruido_final=None,
                 early_stopping=False, patience=10, min_delta=1e-3, val_split=0.1,
                 ruido_adaptativo=False, track_entanglement=False,
                 otimizador='adam', funcao_custo='mse', detectar_barren=False,
                 max_grad_norm=1.0):
        """
        Args:
            n_qubits: Número de qubits (2-20)
            n_camadas: Profundidade do circuito (1-10)
            arquitetura: 'basico' ou 'strongly_entangling'
            estrategia_init: 'aleatorio', 'matematico', ou 'quantico'
            tipo_ruido: 'sem_ruido', 'depolarizante', 'amplitude_damping', 'phase_damping'
            nivel_ruido: Taxa de erro (0.0-0.05)
            taxa_aprendizado: Learning rate para Adam (1e-4 a 1e-1)
            n_epocas: Número de épocas de treinamento (10-200)
            batch_size: Tamanho do mini-batch (8-128)
            seed: Semente aleatória para reprodutibilidade
        """
        self.n_qubits = n_qubits
        self.n_camadas = n_camadas
        self.arquitetura = arquitetura
        self.estrategia_init = estrategia_init
        # Permitir uso automático de 'correlated_noise' se tipo_ruido for 'correlated' ou 'correlated_noise'
        if tipo_ruido in ['correlated', 'correlated_noise']:
            self.tipo_ruido = 'correlated_noise'
        else:
            self.tipo_ruido = tipo_ruido
        self.nivel_ruido = nivel_ruido
        self.taxa_aprendizado = taxa_aprendizado
        self.n_epocas = n_epocas
        self.batch_size = batch_size
        self.seed = seed
        # Annealing de ruído
        self.ruido_schedule = ruido_schedule  # 'linear' | 'exponencial' | 'cosine' | None
        self.ruido_inicial = ruido_inicial
        self.ruido_final = ruido_final
        # Early stopping
        self.early_stopping = early_stopping
        self.patience = patience
        self.min_delta = min_delta
        self.val_split = val_split
        self.ruido_adaptativo = ruido_adaptativo
        self.track_entanglement = track_entanglement

        # Funcionalidades avançadas
        self.otimizador = otimizador  # 'adam', 'sgd', 'qng'
        self.funcao_custo = funcao_custo  # 'mse', 'cross_entropy', 'hinge'
        self.detectar_barren = detectar_barren
        self.max_grad_norm = max_grad_norm

        # Histórico de treinamento expandido
        self.historico_ = {
            'custo': [],
            'acuracia_treino': [],
            'epoca': [],
            'nivel_ruido': [],
            'entropia_emaranhamento': [],
            'variancia_gradiente': []
        }

        # Detectores e monitores
        self.detector_plateau_ = DetectorBarrenPlateau() if detectar_barren else None
        self.monitor_emaranhamento_ = MonitorEmaranhamento(n_qubits) if track_entanglement else None

        # Configurar seeds para reprodutibilidade
        np.random.seed(seed)
        try:
            # Nem sempre disponível no shim; ignore se ausente
            pnp.random.seed(seed)  # type: ignore[attr-defined]
        except Exception:
            pass

    def _criar_circuito(self):
        """
        Cria o circuito quântico e inicializa parâmetros.

        Usa PennyLane's default.mixed device para suportar ruído.
        """
        # Dispositivo quântico (simulador de matriz de densidade)
        self.dev_ = qml.device('default.mixed', wires=self.n_qubits)

        # Selecionar arquitetura e calcular número de parâmetros
        circuito_fn, calc_params = ARQUITETURAS[self.arquitetura]
        n_params = calc_params(self.n_qubits, self.n_camadas)

        # Criar modelo de ruído (se especificado)
        modelo_ruido = None
        if self.tipo_ruido != 'sem_ruido':
            modelo_ruido = MODELOS_RUIDO[self.tipo_ruido](self.nivel_ruido)

        # Criar QNode (circuito quântico diferenciável)
        @qml.qnode(self.dev_, interface='autograd')
        def circuit(weights, x, nivel_ruido_runtime=None):
            """
            Quantum circuit definition for VQC classification.
            
            Args:
                weights: Trainable parameters for the quantum circuit
                x: Input data sample to encode
                nivel_ruido_runtime: Runtime noise level override
                
            Returns:
                Expectation value of PauliZ measurement
            """
            return circuito_fn(weights, x, self.n_qubits, self.n_camadas, modelo_ruido, nivel_ruido_runtime)

        self.qnode_ = circuit

        # Inicializar pesos com estratégia escolhida
        self.weights_ = ConstantesFundamentais.inicializar(
            n_params, self.estrategia_init, self.seed
        )

        # Inicializar bias
        self.bias_ = pnp.array(0.0, requires_grad=True)

    def _nivel_ruido_epoca(self, epoca):
        if self.tipo_ruido == 'sem_ruido':
            return 0.0
        # Sem schedule: usar nivel fixo
        if not self.ruido_schedule:
            return self.nivel_ruido
        nE, ri, rf = max(1, self.n_epocas), (self.ruido_inicial if self.ruido_inicial is not None else self.nivel_ruido), (self.ruido_final if self.ruido_final is not None else 0.001)
        t = epoca / max(1, nE - 1)
        if self.ruido_schedule == 'linear':
            return rf + (ri - rf) * (1 - t)
        if self.ruido_schedule == 'exponencial':
            tau = max(1, nE / 3)
            return rf + (ri - rf) * np.exp(-epoca / tau)
        if self.ruido_schedule == 'cosine':
            return rf + (ri - rf) * 0.5 * (1 + np.cos(np.pi * t))
        return self.nivel_ruido

    def _funcao_custo(self, weights, bias, X, y, nivel_ruido_runtime=None):
        """Função de custo configurável (MSE, Cross-Entropy ou Hinge)."""
        predicoes = pnp.array([self.qnode_(weights, x, nivel_ruido_runtime) + bias for x in X])

        # Selecionar função de custo
        if self.funcao_custo == 'mse':
            # Usar pnp.mean para manter grafo de autograd
            diff = pnp.array(y) - predicoes
            return pnp.mean(diff ** 2)  # type: ignore[attr-defined]
        elif self.funcao_custo == 'cross_entropy':
            return FuncaoCustoAvancada.cross_entropy(predicoes, y)
        elif self.funcao_custo == 'hinge':
            return FuncaoCustoAvancada.hinge(predicoes, y)
        else:
            diff = pnp.array(y) - predicoes
            return pnp.mean(diff ** 2)  # type: ignore[attr-defined]

    def fit(self, X, y):
        """
        Treina o classificador.

        Args:
            X: Dados de treinamento (n_samples, n_features)
            y: Labels (n_samples,)

        Returns:
            self (para compatibilidade scikit-learn)
        """
        # Codificar labels como ±1
        self.label_encoder_ = LabelEncoder()
        y_le = np.asarray(self.label_encoder_.fit_transform(y), dtype=int)
        y_encoded = (y_le * 2) - 1  # type: ignore[operator]
        self.classes_ = self.label_encoder_.classes_

        # Criar circuito e inicializar parâmetros
        self._criar_circuito()

        # Split de validação (para early stopping)
        X_arr, y_arr = np.asarray(X), np.asarray(y_encoded)
        if self.early_stopping and self.val_split > 0:
            n = len(X_arr)
            n_val = max(1, int(n * self.val_split))
            idx = np.random.permutation(n)
            val_idx, train_idx = idx[:n_val], idx[n_val:]
            X_train_es, y_train_es = X_arr[train_idx], y_arr[train_idx]
            X_val_es, y_val_es = X_arr[val_idx], y_arr[val_idx]
        else:
            X_train_es, y_train_es = X_arr, y_arr
            X_val_es = y_val_es = None

        # Criar otimizador configurável
        opt = OtimizadorAvancado.criar(self.otimizador, self.taxa_aprendizado)

        # Treinamento por épocas
        melhor_val = -np.inf
        sem_melhora = 0
        melhor_w, melhor_b = None, None

        for epoca in range(self.n_epocas):
            # Embaralhar dados
            indices = np.random.permutation(len(X_train_es))

            # Mini-batch gradient descent
            nivel_runtime = self._nivel_ruido_epoca(epoca)
            for i in range(0, len(X_train_es), self.batch_size):
                batch_idx = indices[i:i + self.batch_size]
                X_batch = X_train_es[batch_idx]
                y_batch = y_train_es[batch_idx]

                # Atualizar parâmetros (função de custo compatível com autograd)
                def custo_batch(w, b):
                    """
                    Batch cost function for mini-batch gradient descent.
                    
                    Args:
                        w: Weight parameters
                        b: Bias parameter
                        
                    Returns:
                        Mean squared error for the batch
                    """
                    preds = pnp.array([self.qnode_(w, pnp.array(x), nivel_runtime) + b for x in X_batch])
                    # Usar pnp.mean para permitir gradientes
                    return pnp.mean((pnp.array(y_batch) - preds) ** 2)  # type: ignore[attr-defined]

                _step_res = opt.step(custo_batch, self.weights_, self.bias_)
                # Robustez para diferentes retornos
                try:
                    self.weights_, self.bias_ = _step_res  # type: ignore[assignment]
                except Exception:
                    if isinstance(_step_res, (list, tuple)) and len(_step_res) >= 2:
                        self.weights_, self.bias_ = _step_res[0], _step_res[1]
                    else:
                        pass

            # Registrar histórico
            custo_val = self._funcao_custo(
                self.weights_, self.bias_, pnp.array(X_train_es), pnp.array(y_train_es), nivel_runtime
            )
            try:
                custo = float(custo_val)
            except Exception:
                custo = float(pnp.asarray(custo_val))
            acuracia = self.score(X, y)

            self.historico_['custo'].append(custo)
            self.historico_['acuracia_treino'].append(acuracia)
            self.historico_['epoca'].append(epoca)
            self.historico_['nivel_ruido'].append(nivel_runtime)

            # Monitorar gradientes (detecção de barren plateau)
            if self.detector_plateau_:
                try:
                    gradientes = qml.grad(self._funcao_custo)(
                        self.weights_, self.bias_,
                        pnp.array(X_train_es[:5]), pnp.array(y_train_es[:5]),
                        nivel_runtime
                    )
                    if isinstance(gradientes, (list, tuple)) and len(gradientes) > 0:
                        grad_array = np.array(gradientes[0]).flatten()
                    else:
                        grad_array = np.array([])
                    variancia_grad = float(np.var(grad_array))
                    self.historico_['variancia_gradiente'].append(variancia_grad)

                    if self.detector_plateau_.detectar(grad_array):
                        logger.warning(f"Época {epoca}: Barren Plateau detectado (var={variancia_grad:.2e})")
                except Exception:
                    self.historico_['variancia_gradiente'].append(0.0)
            else:
                self.historico_['variancia_gradiente'].append(0.0)

            # Monitorar emaranhamento
            if self.monitor_emaranhamento_:
                try:
                    # Criar QNode que retorna estado
                    @qml.qnode(self.dev_, interface='autograd')
                    def estado_qnode(weights, x):
                        """
                        Quantum node that returns density matrix for entanglement measurement.
                        
                        Args:
                            weights: Circuit weights
                            x: Input data sample
                            
                        Returns:
                            Density matrix of the first qubit
                        """
                        circuito_fn, _ = ARQUITETURAS[self.arquitetura]
                        circuito_fn(weights, x, self.n_qubits, self.n_camadas, None, None)
                        return qml.density_matrix(wires=[0])

                    # Calcular estado do primeiro qubit
                    x_sample = pnp.array(X_train_es[0])
                    rho_0 = estado_qnode(self.weights_, x_sample)

                    # Calcular entropia
                    entropia = self.monitor_emaranhamento_.calcular_entropia_von_neumann(rho_0)
                    self.historico_['entropia_emaranhamento'].append(float(entropia))
                except Exception:
                    self.historico_['entropia_emaranhamento'].append(0.0)
            else:
                self.historico_['entropia_emaranhamento'].append(0.0)

            # Early stopping
            if self.early_stopping and X_val_es is not None and y_val_es is not None:
                ac_val = np.mean(self.predict(X_val_es) == self.label_encoder_.inverse_transform(((y_val_es + 1)//2).astype(int)))
                if ac_val > melhor_val + self.min_delta:
                    melhor_val = ac_val
                    sem_melhora = 0
                    melhor_w, melhor_b = self.weights_.copy(), float(self.bias_)
                else:
                    sem_melhora += 1
                if sem_melhora >= self.patience:
                    # Restaurar melhor
                    if (melhor_w is not None) and (melhor_b is not None):
                        self.weights_ = melhor_w
                        self.bias_ = pnp.array(melhor_b, requires_grad=True)
                    break

        return self

    def predict(self, X):
        """
        Prediz classes para novos dados.

        Args:
            X: Dados de teste (n_samples, n_features)

        Returns:
            Predições de classe (n_samples,)
        """
        # Obter predições do circuito quântico
        predicoes = np.array([
            float(self.qnode_(self.weights_, pnp.array(x)) + self.bias_)
            for x in X
        ])

        # Converter de ±1 para classes originais
        predicoes_classe = ((np.sign(predicoes) + 1) // 2).astype(int)
        return self.label_encoder_.inverse_transform(predicoes_classe)

    def score(self, X, y, sample_weight=None):
        """
        Calcula acurácia.

        Args:
            X: Dados
            y: Labels verdadeiros

        Returns:
            Acurácia (0.0 a 1.0)
        """
        return np.mean(self.predict(X) == y)


# ============================================================================
# MÓDULO 5: GERENCIAMENTO DE DATASETS
# ============================================================================\n\nprint('✓ ClassificadorVQC definido!')

## 7. Carregamento de Datasets\n\n### 💡 Para Iniciantes\nTestamos 5 conjuntos de dados diferentes para verificar se o ruído quântico\nrealmente ajuda em situações variadas.\n\n### 🎓 Para Especialistas\nDatasets do scikit-learn com preprocessamento rigoroso:\n- **Moons**: classificação não-linear 2D\n- **Circles**: classificação não-linear concêntrica\n- **Iris**: multiclasse clássico (3 classes, 4 features)\n- **Breast Cancer**: diagnóstico binário (30 features)\n- **Wine**: multiclasse (3 classes, 13 features)\n\nPreprocessamento: StandardScaler + train/test split (80/20) com seed fixo.

In [ ]:
def carregar_datasets(seed=42):
    """
    Carrega 4 datasets de benchmark para classificação binária.

    Referências:
    - Moons, Circles: Scikit-learn (Pedregosa et al., 2011)
    - Iris: Fisher (1936), UCI ML Repository
    - Breast Cancer: Wolberg et al. (1995), UCI ML Repository

    Returns:
        Dict com 4 datasets, cada um contendo X_train, X_test, y_train, y_test
    """
    datasets = {}
    scaler = StandardScaler()

    # Dataset 1: Moons (não-linear, duas luas entrelaçadas)
    X, y = sk_datasets.make_moons(n_samples=400, noise=0.1, random_state=seed)
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )
    datasets['moons'] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'descricao': 'Duas luas entrelaçadas (não-linear)'
    }

    # Dataset 2: Circles (não-linear, círculos concêntricos)
    X, y = sk_datasets.make_circles(
        n_samples=400, noise=0.1, factor=0.5, random_state=seed
    )
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )
    datasets['circles'] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'descricao': 'Círculos concêntricos (não-linear)'
    }

    # Dataset 3: Iris (linear, 2 primeiras classes)
    from sklearn.utils import Bunch as SklearnBunch
    iris_data: SklearnBunch = sk_datasets.load_iris()  # type: ignore[assignment]
    X, y = iris_data.data, iris_data.target
    mask = y < 2  # Apenas setosa e versicolor
    X, y = X[mask], y[mask]
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )
    datasets['iris'] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'descricao': 'Flores Iris (linear, 4 features)'
    }

    # Dataset 4: Breast Cancer (alta dimensão, 30 features)
    cancer_data: SklearnBunch = sk_datasets.load_breast_cancer()  # type: ignore[assignment]
    X, y = cancer_data.data, cancer_data.target
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )
    datasets['breast_cancer'] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'descricao': 'Diagnóstico de câncer (30 features)'
    }

    # Dataset 5: Wine (features químicas, 2 primeiras classes)
    wine_data: SklearnBunch = sk_datasets.load_wine()  # type: ignore[assignment]
    X, y = wine_data.data, wine_data.target
    mask = y < 2  # Apenas classes 0 e 1
    X, y = X[mask], y[mask]
    X = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed, stratify=y
    )
    datasets['wine'] = {
        'X_train': X_train, 'X_test': X_test,
        'y_train': y_train, 'y_test': y_test,
        'descricao': 'Vinhos (13 features químicas)'
    }

    return datasets


# ============================================================================
# MÓDULO 6: EXECUÇÃO DE EXPERIMENTOS
# ============================================================================\n\nprint('✓ Função carregar_datasets definida!')

## 8. Grid Search de Hiperparâmetros\n\n### 💡 Para Iniciantes\nGrid search testa sistematicamente todas as combinações de parâmetros\npara encontrar a melhor configuração.\n\n### 🎓 Para Especialistas\nBusca exaustiva no espaço de hiperparâmetros:\n- **Arquiteturas**: 9 variantes de circuitos\n- **Inicializações**: 3 estratégias (aleatório, Xavier, He)\n- **Tipos de ruído**: 10 modelos + baseline sem ruído\n- **Níveis de ruído**: scan logarítmico de 0.0001 a 0.1\n- **Datasets**: 5 conjuntos de dados\n\nTotal: ~8,280 experimentos controlados com 3 seeds para robustez estatística.

In [ ]:
def executar_grid_search(datasets, n_epocas=15, verbose=True, pasta_resultados=None):
    """
    Executa grid search completo sobre todas as configurações.

    Grid:
    - 5 datasets (moons, circles, iris, breast_cancer, wine)
    - 8 arquiteturas
    - 5 estratégias de inicialização
    - 5 tipos de ruído (depolarizing, amplitude_damping, phase_damping, crosstalk, thermal/correlated)
    - 3 níveis de ruído
    - 4 schedules de ruído (linear, exponencial, cosseno, adaptativo)
    Total: 5 × 8 × 5 × 5 × 3 × 4 = 12.000 configurações
    # Observação: No código, 'thermal' é tratado como ruído correlacionado para consistência com os documentos.

    Args:
        datasets: Dict de datasets
        n_epocas: Número de épocas de treinamento
        verbose: Se True, imprime progresso

    Returns:
        DataFrame com todos os resultados
    """
    # Pasta para granularidade máxima
    pasta_individual = None
    if pasta_resultados is None:
        pasta_resultados = os.path.join(os.getcwd(), f"resultados_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
    os.makedirs(pasta_resultados, exist_ok=True)
    pasta_individual = os.path.join(pasta_resultados, 'experimentos_individuais')
    os.makedirs(pasta_individual, exist_ok=True)
    # Placeholders para Pylance
    from typing import Any
    metadata: Dict[str, Any] = {}
    metadata_path: Optional[str] = None

    # Definir grid de hiperparâmetros
    quick = os.environ.get('VQC_QUICK', '0') == '1'
    if quick:
        grid = {
            'arquitetura': list(ARQUITETURAS.keys()),
            'estrategia_init': ['matematico', 'fibonacci_spiral'],
            'tipo_ruido': ['sem_ruido', 'depolarizante'],
            'nivel_ruido': [0.0, 0.0025, 0.005, 0.0075, 0.01]
        }
    else:
        grid = {
            'arquitetura': list(ARQUITETURAS.keys()),
            'estrategia_init': ['matematico', 'quantico', 'aleatorio', 'fibonacci_spiral'],
            'tipo_ruido': ['sem_ruido', 'depolarizante', 'amplitude_damping', 'phase_damping', 'crosstalk', 'correlacionado'],
            'nivel_ruido': [0.0, 0.0025, 0.005, 0.0075, 0.01, 0.0125, 0.015, 0.0175, 0.02]
        }

    resultados = []
    n_seeds = 5
    seed_list = [42 + i for i in range(n_seeds)]
    contador = 0  # Inicializar contador

    # Calcular total de configurações
    total_configs = len(grid['arquitetura']) * len(grid['estrategia_init']) * len(grid['tipo_ruido']) * len(grid['nivel_ruido'])
    # Ajustar para configs válidas (sem_ruido só com nivel 0)
    configs_invalidas = len(grid['arquitetura']) * len(grid['estrategia_init']) * (len(grid['nivel_ruido']) - 1)  # sem_ruido com nivel > 0
    total_configs -= configs_invalidas

    # Geração de README e metadata.json após grid/seed_list definidos
    if pasta_resultados is not None:
        readme_path = os.path.join(pasta_resultados, 'README_grid_search.md')
        metadata_path = os.path.join(pasta_resultados, 'metadata_grid_search.json')
        with open(readme_path, 'w', encoding='utf-8') as f:
            f.write(
                "# Resultados do Grid Search VQC\n\n"
                f"- Data de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                f"- Parâmetros do grid: {grid}\n"
                f"- Seeds: {seed_list}\n\n"
                "Todos os experimentos, circuitos e gráficos estão organizados nesta pasta.\n"
                "O arquivo `resultados_completos_artigo.csv` contém todos os resultados consolidados.\n"
            )
        metadata = {
            'tipo': 'grid_search',
            'data_execucao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'parametros_grid': grid,
            'seeds': seed_list,
            'arquivos_gerados': [],
            'csv_consolidado': None,
            'readme': readme_path
        }

    for nome_dataset, dataset in datasets.items():
        _X_train, _y_train = dataset['X_train'], dataset['y_train']
        _X_test, _y_test = dataset['X_test'], dataset['y_test']

        for arq in grid['arquitetura']:
            for init in grid['estrategia_init']:
                for ruido in grid['tipo_ruido']:
                    for nivel in grid['nivel_ruido']:
                        if ruido == 'sem_ruido' and nivel > 0:
                            continue
                        for seed in seed_list:
                            contador += 1
                            # Log detalhado de todos os parâmetros
                            if verbose:
                                logger.info(
                                    f"[{contador:3d}/{total_configs * n_seeds}] "
                                    f"Dataset: {nome_dataset} | Seed: {seed} | Qubits: 4 | Camadas: 2 | "
                                    f"Arquitetura: {arq} | Init: {init} | Ruído: {ruido} | Nível: {nivel:.4f}"
                                )
                                logger.info(
                                    f"Constantes: π={ConstantesFundamentais.PI:.5f}, e={ConstantesFundamentais.E:.5f}, φ={ConstantesFundamentais.PHI:.5f}, ℏ={ConstantesFundamentais.HBAR:.2e}, α={ConstantesFundamentais.ALPHA:.5f}, R∞={ConstantesFundamentais.RYDBERG:.2f}"
                                )
                            try:
                                tempo_inicio = time.time()
                                # Criar e treinar VQC
                                vqc = ClassificadorVQC(
                                    n_qubits=4,
                                    n_camadas=2,
                                    arquitetura=arq,
                                    estrategia_init=init,
                                    tipo_ruido=ruido,
                                    nivel_ruido=nivel,
                                    taxa_aprendizado=0.01,
                                    n_epocas=n_epocas,
                                    batch_size=32,
                                    seed=seed,
                                    # Insights: ruído com annealing quando nível > 0
                                    ruido_schedule=('cosine' if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    ruido_inicial=(nivel if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    ruido_final=(0.001 if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    # Early stopping leve para acelerar
                                    early_stopping=True, patience=5, min_delta=1e-3, val_split=0.1
                                )
                                vqc.fit(dataset['X_train'], dataset['y_train'])
                                tempo_total = time.time() - tempo_inicio
                                # Calcular métricas
                                acuracia_treino = vqc.score(dataset['X_train'], dataset['y_train'])
                                acuracia_teste = vqc.score(dataset['X_test'], dataset['y_test'])
                                gap_treino_teste = acuracia_treino - acuracia_teste
                                # Matriz de confusão
                                y_pred = vqc.predict(dataset['X_test'])
                                cm = confusion_matrix(dataset['y_test'], y_pred)
                                # Armazenar resultados
                                resultado = {
                                    'dataset': nome_dataset,
                                    'arquitetura': arq,
                                    'estrategia_init': init,
                                    'tipo_ruido': ruido,
                                    'nivel_ruido': nivel,
                                    'n_qubits': 4,
                                    'n_camadas': 2,
                                    'acuracia_treino': acuracia_treino,
                                    'acuracia_teste': acuracia_teste,
                                    'gap_treino_teste': gap_treino_teste,
                                    'tempo_segundos': tempo_total,
                                    'custo_final': vqc.historico_['custo'][-1],
                                    'cm_tn': cm[0,0],
                                    'cm_fp': cm[0,1],
                                    'cm_fn': cm[1,0],
                                    'cm_tp': cm[1,1],
                                    'seed': seed
                                }
                                resultados.append(resultado)
                                # Salvar cada experimento individualmente em CSV
                                if pasta_individual is not None:
                                    id_exp = f"exp_{contador:05d}"
                                    df_exp = pd.DataFrame([resultado])
                                    csv_exp_path = os.path.join(pasta_individual, f"{id_exp}.csv")
                                    df_exp.to_csv(csv_exp_path, index=False)
                                # Bloco duplicado removido (já salvo em resultado)
                                if verbose:
                                    logger.info(
                                        f"  ✓ Acurácia: {acuracia_teste:.4f} | "
                                        f"Gap: {gap_treino_teste:+.4f} | "
                                        f"Tempo: {tempo_total:.1f}s"
                                    )

                                # Salvar circuito desenhado em PNG
                                if pasta_resultados is not None:
                                    try:
                                        import pennylane as qml
                                        import matplotlib
                                        matplotlib.use('Agg')
                                        import matplotlib.pyplot as plt

                                        # Criar pasta para circuitos se não existir
                                        pasta_circuitos = os.path.join(pasta_resultados, "circuitos")
                                        os.makedirs(pasta_circuitos, exist_ok=True)

                                        # Desenhar circuito usando qml.draw_mpl
                                        fig_circ, ax_circ = qml.draw_mpl(vqc.qnode_, decimals=2)(
                                            vqc.weights_,
                                            dataset['X_train'][0],
                                            None  # nivel_ruido_runtime
                                        )

                                        circ_png_filename = f"circuito_{nome_dataset}_seed{seed}_{arq}_{init}_{ruido}_nivel{nivel:.4f}.png"
                                        circ_png_path = os.path.join(pasta_circuitos, circ_png_filename)
                                        plt.savefig(circ_png_path, dpi=150, bbox_inches='tight', facecolor='white')
                                        try:
                                            import matplotlib.pyplot as _plt
                                            # Alguns stubs reportam tipo impreciso para draw_mpl; force close seguro
                                            try:
                                                from matplotlib.figure import Figure as _MplFigure
                                            except Exception:
                                                _MplFigure = object  # type: ignore[assignment]
                                            if isinstance(fig_circ, _MplFigure):
                                                _plt.close(fig_circ)  # type: ignore[arg-type]
                                            else:
                                                _plt.close('all')
                                        except Exception:
                                            try:
                                                plt.close('all')
                                            except Exception:
                                                pass

                                        if verbose:
                                            logger.info(f"    → Circuito salvo: {circ_png_filename}")
                                    except Exception as e:
                                        logger.warning(f"Falha ao salvar PNG do circuito: {e}")

                                    # Salvar gráfico 3D de gradientes (Barren Plateaus)
                                    if 'variancia_gradiente' in vqc.historico_ and len(vqc.historico_['variancia_gradiente']) > 0:
                                        try:
                                            import matplotlib
                                            matplotlib.use('Agg')
                                            import matplotlib.pyplot as plt
                                            import numpy as np

                                            # Criar pasta para barren plateaus se não existir
                                            pasta_barren = os.path.join(pasta_resultados, "barren_plateaus")
                                            os.makedirs(pasta_barren, exist_ok=True)

                                            epocas = vqc.historico_.get('epoca', [])
                                            variancias = vqc.historico_.get('variancia_gradiente', [])
                                            custos = vqc.historico_.get('custo', [])

                                            # Se não houver dados de época, gerar sequência simples
                                            if not epocas or len(epocas) != len(variancias):
                                                epocas = list(range(1, len(variancias)+1))

                                            if len(epocas) == len(variancias) == len(custos) and len(epocas) > 0:
                                                fig = plt.figure(figsize=(10, 8))
                                                ax = fig.add_subplot(111, projection='3d')

                                                # Garantir dtype float para compatibilidade com matplotlib
                                                X = np.array(epocas, dtype=float)
                                                Y = np.array(variancias, dtype=float)
                                                Z = np.array(custos, dtype=float)

                                                scatter = ax.scatter(xs=X, ys=Y, zs=Z, c=Z, cmap='viridis', s=50, alpha=0.8)  # type: ignore[call-arg]

                                                ax.set_title(
                                                    f'Barren Plateau Analysis\n{arq} | {init} | {ruido} (γ={nivel:.4f})',
                                                    fontsize=14, fontfamily='serif'
                                                )
                                                ax.set_xlabel('Época', fontsize=12, fontfamily='serif')
                                                ax.set_ylabel('Var(Gradiente)', fontsize=12, fontfamily='serif')
                                                ax.set_zlabel('Custo', fontsize=12, fontfamily='serif')

                                                cbar = plt.colorbar(scatter, ax=ax, label='Custo', shrink=0.8, pad=0.1)
                                                cbar.ax.tick_params(labelsize=10)

                                                plt.tight_layout()

                                                barren_filename = f"barren3d_{nome_dataset}_seed{seed}_{arq}_{init}_{ruido}_nivel{nivel:.4f}.png"
                                                barren_path = os.path.join(pasta_barren, barren_filename)
                                                plt.savefig(barren_path, dpi=150, bbox_inches='tight', facecolor='white')
                                                # Fechamento explícito da figura para evitar warning de tipo do Pylance
                                                try:
                                                    import matplotlib.pyplot as _plt
                                                    _plt.close(fig)
                                                except Exception:
                                                    try:
                                                        plt.close('all')
                                                    except Exception:
                                                        pass
                                                if verbose:
                                                    logger.info(f"    → Barren plateau 3D salvo: {barren_filename}")
                                            else:
                                                logger.warning("Dados insuficientes ou incompatíveis para gerar gráfico 3D dos gradientes.")
                                        except Exception as e:
                                            logger.warning(f"Falha ao salvar gráfico 3D barren plateau: {e}")

                            except Exception as e:
                                if verbose:
                                    logger.warning(f"  ✗ Erro: {str(e)[:50]}")

    total_configs = (len(grid['arquitetura']) * len(grid['estrategia_init']) *
                    len(grid['tipo_ruido']) * len(grid['nivel_ruido']))

    if verbose:
        logger.info(f"Total de configurações: {total_configs} por dataset")
        logger.info(f"Total geral: {total_configs * len(datasets)} experimentos\n")

    contador = 0

    # Iterar sobre datasets
    for nome_dataset, dataset in datasets.items():
        if verbose:
            logger.info(f"\n{'='*80}")
            logger.info(f" DATASET: {nome_dataset.upper()}")
            logger.info(f" {dataset['descricao']}")
            logger.info(f"{'='*80}\n")
        # Iterar sobre grid
        for arq in grid['arquitetura']:
            for init in grid['estrategia_init']:
                for ruido in grid['tipo_ruido']:
                    for nivel in grid['nivel_ruido']:
                        # Pular combinações inválidas (sem_ruido com nível > 0)
                        if ruido == 'sem_ruido' and nivel > 0:
                            continue
                        for seed in seed_list:
                            contador += 1
                            # Log detalhado de todos os parâmetros
                            if verbose:
                                logger.info(
                                    f"[{contador:3d}/{total_configs * n_seeds}] "
                                    f"Dataset: {nome_dataset} | Seed: {seed} | Qubits: 4 | Camadas: 2 | "
                                    f"Arquitetura: {arq} | Init: {init} | Ruído: {ruido} | Nível: {nivel:.4f}"
                                )
                                logger.info(
                                    f"Constantes: π={ConstantesFundamentais.PI:.5f}, e={ConstantesFundamentais.E:.5f}, φ={ConstantesFundamentais.PHI:.5f}, ℏ={ConstantesFundamentais.HBAR:.2e}, α={ConstantesFundamentais.ALPHA:.5f}, R∞={ConstantesFundamentais.RYDBERG:.2f}"
                                )
                            try:
                                tempo_inicio = time.time()
                                # Criar e treinar VQC
                                vqc = ClassificadorVQC(
                                    n_qubits=4,
                                    n_camadas=2,
                                    arquitetura=arq,
                                    estrategia_init=init,
                                    tipo_ruido=ruido,
                                    nivel_ruido=nivel,
                                    taxa_aprendizado=0.01,
                                    n_epocas=n_epocas,
                                    batch_size=32,
                                    seed=seed,
                                    # Insights: ruído com annealing quando nível > 0
                                    ruido_schedule=('cosine' if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    ruido_inicial=(nivel if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    ruido_final=(0.001 if (ruido != 'sem_ruido' and nivel > 0) else None),
                                    # Early stopping leve para acelerar
                                    early_stopping=True, patience=5, min_delta=1e-3, val_split=0.1
                                )
                                vqc.fit(dataset['X_train'], dataset['y_train'])
                                tempo_total = time.time() - tempo_inicio
                                # Calcular métricas
                                acuracia_treino = vqc.score(dataset['X_train'], dataset['y_train'])
                                acuracia_teste = vqc.score(dataset['X_test'], dataset['y_test'])
                                gap_treino_teste = acuracia_treino - acuracia_teste
                                # Matriz de confusão
                                y_pred = vqc.predict(dataset['X_test'])
                                cm = confusion_matrix(dataset['y_test'], y_pred)
                                # Armazenar resultados
                                resultados.append({
                                    'dataset': nome_dataset,
                                    'arquitetura': arq,
                                    'estrategia_init': init,
                                    'tipo_ruido': ruido,
                                    'nivel_ruido': nivel,
                                    'n_qubits': 4,
                                    'n_camadas': 2,
                                    'acuracia_treino': acuracia_treino,
                                    'acuracia_teste': acuracia_teste,
                                    'gap_treino_teste': gap_treino_teste,
                                    'tempo_segundos': tempo_total,
                                    'custo_final': vqc.historico_['custo'][-1],
                                    'cm_tn': cm[0,0],
                                    'cm_fp': cm[0,1],
                                    'cm_fn': cm[1,0],
                                    'cm_tp': cm[1,1],
                                    'seed': seed
                                })
                                if verbose:
                                    logger.info(
                                        f"  ✓ Acurácia: {acuracia_teste:.4f} | "
                                        f"Gap: {gap_treino_teste:+.4f} | "
                                        f"Tempo: {tempo_total:.1f}s"
                                    )
                            except Exception as e:
                                if verbose:
                                    logger.warning(f"  ✗ Erro: {str(e)[:50]}")

    # Adicionar baselines clássicos (SVM e Random Forest)
    for nome_dataset, dataset in datasets.items():
        if verbose:
            logger.info(f"\n{'='*80}")
            logger.info(f" BASELINES CLÁSSICOS: {nome_dataset.upper()}")
            logger.info(f"{'='*80}")
        # SVM (RBF)
        try:
            clf_svm = SVC(kernel='rbf', probability=True, random_state=42)
            clf_svm.fit(dataset['X_train'], dataset['y_train'])
            acuracia_treino = clf_svm.score(dataset['X_train'], dataset['y_train'])
            acuracia_teste = clf_svm.score(dataset['X_test'], dataset['y_test'])
            y_pred = clf_svm.predict(dataset['X_test'])
            cm = confusion_matrix(dataset['y_test'], y_pred)
            resultados.append({
                'dataset': nome_dataset,
                'arquitetura': 'SVM',
                'estrategia_init': '-',
                'tipo_ruido': 'classico',
                'nivel_ruido': 0.0,
                'n_qubits': 0,
                'n_camadas': 0,
                'acuracia_treino': acuracia_treino,
                'acuracia_teste': acuracia_teste,
                'gap_treino_teste': acuracia_treino - acuracia_teste,
                'tempo_segundos': 0.0,
                'custo_final': 0.0,
                'cm_tn': cm[0,0],
                'cm_fp': cm[0,1],
                'cm_fn': cm[1,0],
                'cm_tp': cm[1,1],
                'seed': 42
            })
            if verbose:
                logger.info(f"  ✓ SVM (RBF): Acurácia teste = {acuracia_teste:.4f}")
        except Exception as e:
            if verbose:
                logger.warning(f"  ✗ Erro SVM: {str(e)[:50]}")
        # Random Forest
        try:
            clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
            clf_rf.fit(dataset['X_train'], dataset['y_train'])
            acuracia_treino = clf_rf.score(dataset['X_train'], dataset['y_train'])
            acuracia_teste = clf_rf.score(dataset['X_test'], dataset['y_test'])
            y_pred = clf_rf.predict(dataset['X_test'])
            cm = confusion_matrix(dataset['y_test'], y_pred)
            resultados.append({
                'dataset': nome_dataset,
                'arquitetura': 'RandomForest',
                'estrategia_init': '-',
                'tipo_ruido': 'classico',
                'nivel_ruido': 0.0,
                'n_qubits': 0,
                'n_camadas': 0,
                'acuracia_treino': acuracia_treino,
                'acuracia_teste': acuracia_teste,
                'gap_treino_teste': acuracia_treino - acuracia_teste,
                'tempo_segundos': 0.0,
                'custo_final': 0.0,
                'cm_tn': cm[0,0],
                'cm_fp': cm[0,1],
                'cm_fn': cm[1,0],
                'cm_tp': cm[1,1],
                'seed': 42
            })
            if verbose:
                logger.info(f"  ✓ Random Forest: Acurácia teste = {acuracia_teste:.4f}")
        except Exception as e:
            if verbose:
                logger.warning(f"  ✗ Erro RF: {str(e)[:50]}")

    df_resultados = pd.DataFrame(resultados)
    # Salvar CSV consolidado e atualizar metadata
    if pasta_resultados is not None:
        csv_path = os.path.join(pasta_resultados, 'resultados_completos_artigo.csv')
        df_resultados.to_csv(csv_path, index=False)
        # Adicionar granularidade máxima ao metadata
        metadata['csv_consolidado'] = csv_path
        metadata['csvs_individuais'] = [os.path.join('experimentos_individuais', f) for f in os.listdir(pasta_individual) if f.endswith('.csv')]
        # Atualizar lista de arquivos
        metadata['arquivos_gerados'] = [f for f in os.listdir(pasta_resultados) if os.path.isfile(os.path.join(pasta_resultados, f))]
        # Salvar metadata.json
        if metadata_path:
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False, default=str)
    if verbose:
        logger.info(f"\n{'='*80}")
        logger.info(f" ✓ GRID SEARCH CONCLUÍDO: {len(df_resultados)} experimentos")
        logger.info(f"{'='*80}\n")
    return df_resultados


# ============================================================================
# MÓDULO 7: ANÁLISES ESTATÍSTICAS
# ============================================================================\n\nprint('✓ Função executar_grid_search definida!')

## 9. Análises Estatísticas Avançadas (QUALIS A1)\n\n### 💡 Para Iniciantes\nUsamos estatística rigorosa para provar que o ruído realmente ajuda,\nnão é apenas sorte ou acaso.\n\n### 🎓 Para Especialistas\nPipeline estatístico completo conforme padrões QUALIS A1:\n\n1. **ANOVA**: F-test para diferenças entre grupos\n2. **Post-hoc tests**: Bonferroni, Scheffé, Tukey HSD\n3. **Effect sizes**: \n   - Cohen's d: $(\\mu_1 - \\mu_2) / s_{pooled}$\n   - Glass's Δ: $(\\mu_1 - \\mu_2) / s_{control}$\n   - Hedges' g: Cohen's d com correção para pequenas amostras\n4. **Intervalos de confiança**: 95% via bootstrap\n5. **Testes de normalidade**: Shapiro-Wilk\n6. **Homogeneidade de variâncias**: Levene

In [ ]:
def executar_analises_estatisticas(df, verbose=True, pasta_resultados=None):
    """Executa análises estatísticas principais do artigo."""
    import os

    # Inicializar metadata
    analise_meta: dict[str, Any] = {
        'tipo': 'analises_estatisticas',
        'data_execucao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'arquivos_gerados': [],
        'csvs': {}
    }
    metadata_path: Optional[str] = None
    pasta_individual: Optional[str] = None

    if pasta_resultados is not None:
        pasta_individual = os.path.join(pasta_resultados, 'analises_individuais')
        os.makedirs(pasta_individual, exist_ok=True)
        os.makedirs(pasta_resultados, exist_ok=True)
        # Forense: README e metadata
        readme_path = os.path.join(pasta_resultados, 'README_analises_estatisticas.md')
        metadata_path = os.path.join(pasta_resultados, 'metadata_analises_estatisticas.json')
        with open(readme_path, 'w', encoding='utf-8') as f:
            f.write(
                "# Análises Estatísticas\n\n"
                f"- Data de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                "- Conteúdo: ANOVA, comparação de inicializações, overfitting, effect sizes, post-hoc.\n"
            )

    if verbose:
        logger.info("="*80)
        logger.info(" ANÁLISES ESTATÍSTICAS")
        logger.info("="*80)

    analises = {}

    # 1. ANOVA 2-way: Noise × Dataset
    if verbose:
        logger.info("\n1. ANOVA 2-WAY: Noise Level × Dataset")
        logger.info("-"*80)

    # Verificar se há dados
    if len(df) == 0:
        logger.warning("Nenhum resultado disponível para análise!")
        return {'erro': 'Sem dados'}

    df_anova = df.copy()
    df_anova['nivel_ruido_cat'] = df_anova['nivel_ruido'].astype(str)

    # Proteger: ANOVA 2-way requer pelo menos 2 níveis em cada fator
    if df_anova['nivel_ruido_cat'].nunique() >= 2 and df_anova['dataset'].nunique() >= 2:
        model = ols('acuracia_teste ~ C(nivel_ruido_cat) + C(dataset) + C(nivel_ruido_cat):C(dataset)',
                    data=df_anova).fit()
        anova_2way = anova_lm(model, typ=2)
        analises['anova_2way'] = anova_2way
        if verbose:
            print(anova_2way)
    else:
        analises['anova_2way'] = 'Insuficiente para ANOVA 2-way (>=2 níveis por fator)'
        if verbose:
            logger.info('Dados insuficientes para ANOVA 2-way, análise pulada.')

    # 2. Comparação de inicializações
    if verbose:
        logger.info("\n2. COMPARAÇÃO DE INICIALIZAÇÕES")
        logger.info("-"*80)

    # Build aggregation dict based on available columns
    agg_dict = {'acuracia_teste': ['mean', 'std']}
    if 'tempo_segundos' in df.columns:
        agg_dict['tempo_segundos'] = 'mean'
    else:
        if verbose:
            logger.info("  ℹ️ Coluna 'tempo_segundos' não disponível, análise de tempo não será incluída.")

    comp_init = df.groupby('estrategia_init').agg(agg_dict).round(4)

    if verbose:
        print(comp_init)
    # Salvar CSV resumo de inicializações
    if pasta_resultados is not None:
        comp_init_path = os.path.join(pasta_resultados, 'analise_comparacao_inicializacoes.csv')
        try:
            comp_init.to_csv(comp_init_path)
            analise_meta['csvs']['comparacao_inicializacoes'] = comp_init_path
        except Exception:
            pass

    # 2b. Comparação consolidada: VQC vs Baselines Clássicos (SVM/RF)
    try:
        if verbose:
            logger.info("\n2b. COMPARAÇÃO: VQC vs SVM/RF (por dataset)")
            logger.info("-"*80)
        df_q = df[df['tipo_ruido'] != 'classico']
        df_class = df[df['tipo_ruido'] == 'classico']
        # Melhor VQC por dataset (maior acurácia)
        vqc_best = df_q.groupby('dataset')['acuracia_teste'].max().rename('vqc_melhor')
        # VQC sem ruído (média por dataset)
        vqc_sem = df[df['tipo_ruido'] == 'sem_ruido'].groupby('dataset')['acuracia_teste'].mean().rename('vqc_sem_ruido_media')
        # Baselines
        svm = (
            df_class[df_class['arquitetura'] == 'SVM']
            .groupby('dataset')['acuracia_teste']
            .mean()
            .rename('svm')
        )
        rf = (
            df_class[df_class['arquitetura'] == 'RandomForest']
            .groupby('dataset')['acuracia_teste']
            .mean()
            .rename('rf')
        )
        comp = pd.concat([vqc_best, vqc_sem, svm, rf], axis=1)
        # Deltas (podem ser NaN se baseline ausente)
        comp['delta_vqc_svm'] = comp['vqc_melhor'] - comp['svm']
        comp['delta_vqc_rf'] = comp['vqc_melhor'] - comp['rf']
        comp = comp.reset_index()
        if verbose:
            logger.info("Resumo por dataset:")
            logger.info(comp.round(4).to_string(index=False))
        if pasta_resultados is not None:
            comp_path = os.path.join(pasta_resultados, 'comparacao_baselines.csv')
            try:
                comp.to_csv(comp_path, index=False)
                analise_meta['csvs']['comparacao_baselines'] = comp_path
            except Exception:
                pass
    except Exception as e:
        if verbose:
            logger.warning(f"Falha ao gerar comparacao_baselines.csv: {str(e)[:80]}")
    # Salvar DataFrame completo das análises estatísticas
    try:
        df.to_csv(os.path.join(str(pasta_resultados), 'analises_estatisticas_completo.csv'), index=False)
        analise_meta['csvs']['completo'] = os.path.join(str(pasta_resultados), 'analises_estatisticas_completo.csv')
        # Salvar cada análise individualmente em CSV
        if pasta_individual is not None:
            for idx, row in df.iterrows():
                id_analise = f"analise_{idx:05d}"
                df_row = pd.DataFrame([row])
                csv_analise_path = os.path.join(str(pasta_individual), f"{id_analise}.csv")
                df_row.to_csv(csv_analise_path, index=False)
            # Listar CSVs apenas se pasta_individual é válido
            analise_meta['csvs_individuais'] = [os.path.join('analises_individuais', f) for f in os.listdir(str(pasta_individual)) if f.endswith('.csv')]
    except Exception:
        pass

    analises['comparacao_inicializacoes'] = comp_init

    # 3. Análise de overfitting
    if verbose:
        logger.info("\n3. ANÁLISE DE OVERFITTING")
        logger.info("-"*80)

    # Check if required columns exist
    if 'gap_treino_teste' in df.columns and 'tipo_ruido' in df.columns:
        gap_sem_ruido = df[df['tipo_ruido'] == 'sem_ruido']['gap_treino_teste'].mean()
        mask_otimo = (df['tipo_ruido'] == 'depolarizante') & (df['nivel_ruido'] == 0.01)
        gap_com_ruido = df[mask_otimo]['gap_treino_teste'].mean()

        if not np.isnan(gap_sem_ruido) and not np.isnan(gap_com_ruido) and gap_sem_ruido != 0:
            reducao_overfitting = ((gap_sem_ruido - gap_com_ruido) / gap_sem_ruido) * 100
        else:
            reducao_overfitting = 0.0

        if verbose:
            logger.info(f"Gap sem ruído: {gap_sem_ruido:.4f}")
            logger.info(f"Gap com ruído ótimo: {gap_com_ruido:.4f}")
            logger.info(f"Redução de overfitting: {reducao_overfitting:.1f}%")

        analises['overfitting'] = {
            'gap_sem_ruido': gap_sem_ruido,
            'gap_com_ruido': gap_com_ruido,
            'reducao_percent': reducao_overfitting
        }
    else:
        if verbose:
            logger.info("Colunas necessárias não disponíveis para análise de overfitting.")
        analises['overfitting'] = {
            'gap_sem_ruido': np.nan,
            'gap_com_ruido': np.nan,
            'reducao_percent': 0.0
        }

    # 4. Effect Sizes (Cohen's d, Glass's Δ, Hedges' g)
    if verbose:
        logger.info("\n4. EFFECT SIZES")
        logger.info("-"*80)

    sem_ruido = df[df['tipo_ruido'] == 'sem_ruido']['acuracia_teste'].values
    com_ruido = df[df['tipo_ruido'] != 'sem_ruido']['acuracia_teste'].values

    if len(sem_ruido) > 0 and len(com_ruido) > 0:
        cohen_d = TestesEstatisticosAvancados.cohen_d(com_ruido, sem_ruido)
        glass_delta = TestesEstatisticosAvancados.glass_delta(com_ruido, sem_ruido)
        hedges_g = TestesEstatisticosAvancados.hedges_g(com_ruido, sem_ruido)

        if verbose:
            logger.info(f"Cohen's d: {cohen_d:.4f}")
            logger.info(f"Glass's Δ: {glass_delta:.4f}")
            logger.info(f"Hedges' g: {hedges_g:.4f}")

        analises['effect_sizes'] = {
            'cohen_d': cohen_d,
            'glass_delta': glass_delta,
            'hedges_g': hedges_g
        }

    # 5. Testes Post-hoc (Bonferroni)
    if verbose:
        logger.info("\n5. TESTES POST-HOC")
        logger.info("-"*80)

    # Comparar cada tipo de ruído vs. baseline
    tipos_ruido = df['tipo_ruido'].unique()
    p_values = []

    for tipo in tipos_ruido:
        if tipo != 'sem_ruido':
            grupo1 = df[df['tipo_ruido'] == tipo]['acuracia_teste'].values
            grupo2 = df[df['tipo_ruido'] == 'sem_ruido']['acuracia_teste'].values

            if len(grupo1) > 0 and len(grupo2) > 0:
                _, p_val = ttest_ind(grupo1, grupo2)
                p_values.append((tipo, p_val))

    if len(p_values) > 0:
        # Correção de Bonferroni
        p_vals_only = [p for _, p in p_values]
        significantes = TestesEstatisticosAvancados.bonferroni(p_vals_only, alpha=0.05)

        if verbose:
            for (tipo, p_val), sig in zip(p_values, significantes):
                status = "✓ Significativo" if sig else "✗ Não significativo"
                logger.info(f"{tipo:20s}: p={p_val:.4f} {status}")

        analises['posthoc_bonferroni'] = list(zip(p_values, significantes))

    # Persistir metadata
    if pasta_resultados is not None and metadata_path is not None:
        # Atualizar lista de arquivos gerados no diretório
        try:
            analise_meta['arquivos_gerados'] = [f for f in os.listdir(pasta_resultados) if os.path.isfile(os.path.join(pasta_resultados, f))]
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(analise_meta, f, indent=2, ensure_ascii=False, default=str)
        except Exception:
            pass
    return analises


# ============================================================================
# MÓDULO 8: VISUALIZAÇÕES
# ============================================================================\n\nprint('✓ Função executar_analises_estatisticas definida!')

## 10. Geração de Visualizações Científicas\n\n### 💡 Para Iniciantes\nGráficos interativos que mostram claramente como o ruído afeta o desempenho.\n\n### 🎓 Para Especialistas\nVisualizações com padrões de publicação QUALIS A1:\n- Resolução 300 DPI\n- Fonte Times New Roman\n- Barras de erro (SEM × 1.96 para IC 95%)\n- Legendas científicas completas\n- Formato interativo (Plotly) e exportável (PNG/SVG)

In [ ]:
def gerar_visualizacoes(df, salvar=True, pasta_resultados=None):
    """Gera as figuras principais do artigo."""
    import os

    # Inicializar metadata
    viz_meta: dict[str, Any] = {
        'tipo': 'visualizacoes',
        'data_execucao': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'arquivos_gerados': [],
        'figuras': []
    }
    metadata_path: Optional[str] = None
    pasta_individual: Optional[str] = None

    if pasta_resultados is not None:
        pasta_individual = os.path.join(pasta_resultados, 'visualizacoes_individuais')
        os.makedirs(pasta_individual, exist_ok=True)
        os.makedirs(pasta_resultados, exist_ok=True)
        # Forense: README e metadata
        readme_path = os.path.join(pasta_resultados, 'README_visualizacoes.md')
        metadata_path = os.path.join(pasta_resultados, 'metadata_visualizacoes.json')
        os.makedirs(os.path.dirname(readme_path), exist_ok=True)
        os.makedirs(os.path.dirname(metadata_path), exist_ok=True)
        with open(readme_path, 'w', encoding='utf-8') as f:
            f.write(
                "# Visualizações Geradas\n\n"
                f"- Data de execução: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
                "- Figuras: 2, 2b, 3, 3b, 4, 5, 6, 7\n"
            )

    logger.info("="*80)
    logger.info(" GERANDO VISUALIZAÇÕES")
    logger.info("="*80)

    figuras = {}

    # FIGURA 2: Beneficial Noise (PRINCIPAL)
    logger.info("\n" + "="*80)
    logger.info("GERANDO FIGURA 2: BENEFICIAL NOISE ANALYSIS (QUALIS A1)")
    logger.info("="*80)
    logger.info("Specifications: High-resolution (300 DPI), Publication-ready formats")
    logger.info("Formats: HTML (interactive), PNG, PDF, SVG")

    fig2 = px.scatter(
        df, x='nivel_ruido', y='acuracia_teste',
        color='tipo_ruido', facet_col='dataset',
        title="Figure 2: Quantum Noise Impact on Classifier Accuracy (Beneficial Regime Analysis)",
        labels={
            'nivel_ruido': 'Noise Level (γ)', 
            'acuracia_teste': 'Test Accuracy (%)',
            'tipo_ruido': 'Noise Type'
        },
        height=600
    )

    if salvar:
        # Qualis A1: aprimorar layout para publicação científica
        fig2.update_layout(
            font=dict(family='Times New Roman, serif', size=18, color='black'),
            title_font=dict(size=24, family='Times New Roman, serif', color='black', weight="bold"),
            legend_title_font=dict(size=20, family='Times New Roman, serif', color='black', weight="bold"),
            legend_font=dict(size=18, family='Times New Roman, serif', color='black'),
            margin=dict(l=80, r=60, t=100, b=80),
            paper_bgcolor='white',
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor='black',
                borderwidth=1
            )
        )
        fig2.update_traces(
            marker=dict(
                size=8, 
                line=dict(width=1.5, color='black'),
                opacity=0.8
            )
        )
        fig2.update_xaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray', 
            zeroline=False, 
            ticks='outside', 
            tickfont=dict(size=16, family='Times New Roman, serif'),
            linewidth=2,
            linecolor='black',
            mirror=True
        )
        fig2.update_yaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray', 
            zeroline=False, 
            ticks='outside', 
            tickfont=dict(size=16, family='Times New Roman, serif'),
            linewidth=2,
            linecolor='black',
            mirror=True
        )
        # Exportar em alta resolução e formatos científicos (QUALIS A1 standards)
        path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura2_beneficial_noise.html')
        os.makedirs(os.path.dirname(path_html), exist_ok=True)
        fig2.write_html(path_html)
        logger.info(f"  ✓ Saved: {os.path.basename(path_html)} (interactive HTML)")
        
        if pasta_resultados is not None:
            viz_meta['figuras'].append(path_html)
            path_png = os.path.join(pasta_resultados, 'figura2_beneficial_noise.png')
            path_pdf = os.path.join(pasta_resultados, 'figura2_beneficial_noise.pdf')
            path_svg = os.path.join(pasta_resultados, 'figura2_beneficial_noise.svg')
            for p in [path_png, path_pdf, path_svg]:
                os.makedirs(os.path.dirname(p), exist_ok=True)
            # QUALIS A1: Exportar em 300 DPI (scale=3 para 1200x800 = 300 DPI)
            logger.info(f"  ⏳ Exporting high-resolution formats (300 DPI)...")
            fig2.write_image(path_png, format='png', scale=3, width=1600, height=1000)
            fig2.write_image(path_pdf, format='pdf', width=1600, height=1000)
            fig2.write_image(path_svg, format='svg', width=1600, height=1000)
            viz_meta['figuras'] += [path_png, path_pdf, path_svg]
            logger.info(f"  ✓ Saved: {os.path.basename(path_png)} (PNG 300 DPI)")
            logger.info(f"  ✓ Saved: {os.path.basename(path_pdf)} (PDF vector)")
            logger.info(f"  ✓ Saved: {os.path.basename(path_svg)} (SVG vector)")
    logger.info("FIGURA 2: COMPLETED")
    logger.info("="*80)
    figuras['figura2'] = fig2

    # FIGURA 2b: Beneficial Noise com IC95% por grupo (dataset, tipo_ruido, nivel_ruido)
    logger.info("\n" + "="*80)
    logger.info("GERANDO FIGURA 2b: BENEFICIAL NOISE WITH 95% CONFIDENCE INTERVALS")
    logger.info("="*80)
    try:
        df_q = df[df['tipo_ruido'] != 'classico'].copy()
        grp_cols = ['dataset', 'tipo_ruido', 'nivel_ruido']
        df_ci = (
            df_q.groupby(grp_cols)
            .agg(media=('acuracia_teste', 'mean'), desvio=('acuracia_teste', 'std'), n=('acuracia_teste', 'count'))
            .reset_index()
        )
        # Evitar divisão por zero para n<=1
        df_ci['sem'] = df_ci.apply(lambda r: (r['desvio'] / np.sqrt(r['n'])) if r['n'] > 1 and r['desvio'] == r['desvio'] else 0.0, axis=1)
        df_ci['ci95'] = 1.96 * df_ci['sem']
        logger.info(f"  Statistical Summary: {len(df_ci)} data points with 95% CI")
        
        fig2b = px.scatter(
            df_ci, x='nivel_ruido', y='media', color='tipo_ruido', facet_col='dataset',
            error_y='ci95',
            title='Figure 2b: Mean Accuracy ± 95% CI by Noise Level',
            labels={
                'nivel_ruido': 'Noise Level (γ)', 
                'media': 'Mean Test Accuracy (%)',
                'tipo_ruido': 'Noise Type'
            },
            height=600
        )
        # Aparência consistente com QUALIS A1
        fig2b.update_layout(
            font=dict(family='Times New Roman, serif', size=18, color='black'),
            title_font=dict(size=24, family='Times New Roman, serif', color='black', weight="bold"),
            legend_title_font=dict(size=20, family='Times New Roman, serif', color='black', weight="bold"),
            legend_font=dict(size=18, family='Times New Roman, serif', color='black'),
            margin=dict(l=80, r=60, t=100, b=80),
            paper_bgcolor='white', 
            plot_bgcolor='white',
            showlegend=True,
            legend=dict(
                bgcolor='rgba(255,255,255,0.9)',
                bordercolor='black',
                borderwidth=1
            )
        )
        fig2b.update_traces(
            marker=dict(
                size=10, 
                line=dict(width=1.5, color='black'),
                opacity=0.8
            ),
            error_y=dict(thickness=2, width=6)
        )
        fig2b.update_xaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray', 
            zeroline=False, 
            ticks='outside', 
            tickfont=dict(size=16, family='Times New Roman, serif'),
            linewidth=2,
            linecolor='black',
            mirror=True
        )
        fig2b.update_yaxes(
            showgrid=True, 
            gridwidth=1, 
            gridcolor='lightgray', 
            zeroline=False, 
            ticks='outside', 
            tickfont=dict(size=16, family='Times New Roman, serif'),
            linewidth=2,
            linecolor='black',
            mirror=True
        )
        if salvar:
            path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura2b_beneficial_noise_ic95.html')
            os.makedirs(os.path.dirname(path_html), exist_ok=True)
            fig2b.write_html(path_html)
            logger.info(f"  ✓ Saved: {os.path.basename(path_html)} (interactive HTML)")
            
            if pasta_resultados is not None:
                viz_meta['figuras'].append(path_html)
                path_png = os.path.join(pasta_resultados, 'figura2b_beneficial_noise_ic95.png')
                path_pdf = os.path.join(pasta_resultados, 'figura2b_beneficial_noise_ic95.pdf')
                path_svg = os.path.join(pasta_resultados, 'figura2b_beneficial_noise_ic95.svg')
                for p in [path_png, path_pdf, path_svg]:
                    os.makedirs(os.path.dirname(p), exist_ok=True)
                logger.info(f"  ⏳ Exporting high-resolution formats (300 DPI)...")
                fig2b.write_image(path_png, format='png', scale=3, width=1600, height=1000)
                fig2b.write_image(path_pdf, format='pdf', width=1600, height=1000)
                fig2b.write_image(path_svg, format='svg', width=1600, height=1000)
                viz_meta['figuras'] += [path_png, path_pdf, path_svg]
                logger.info(f"  ✓ Saved: {os.path.basename(path_png)} (PNG 300 DPI)")
                logger.info(f"  ✓ Saved: {os.path.basename(path_pdf)} (PDF vector)")
                logger.info(f"  ✓ Saved: {os.path.basename(path_svg)} (SVG vector)")
        figuras['figura2b'] = fig2b
        logger.info("FIGURA 2b: COMPLETED")
        logger.info("="*80)
    except Exception as e:
        logger.warning(f"Não foi possível gerar a Figura 2b (IC95%): {str(e)[:80]}")

    # FIGURA 3: Noise Type Comparison
    logger.info("Gerando Figura 3: Comparação de Tipos de Ruído...")

    fig3 = px.box(
        df, x='tipo_ruido', y='acuracia_teste', color='tipo_ruido',
        title="Figura 3: Comparação de Tipos de Ruído",
        labels={'tipo_ruido': 'Tipo de Ruído', 'acuracia_teste': 'Acurácia no Teste'},
        height=500
    )

    if salvar:
        fig3.update_layout(
            font=dict(family='serif', size=18, color='black'),
            title_font=dict(size=22, family='serif', color='black', weight="bold"),
            legend_title_font=dict(size=18, family='serif', color='black', weight="bold"),
            legend_font=dict(size=16, family='serif', color='black'),
            margin=dict(l=60, r=40, t=80, b=60),
            paper_bgcolor='white',
            plot_bgcolor='white',
        )
        fig3.update_traces(marker=dict(line=dict(width=1, color='black')))
        fig3.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        fig3.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura3_noise_types.html')
        os.makedirs(os.path.dirname(path_html), exist_ok=True)
        fig3.write_html(path_html)
        if pasta_resultados is not None:
            viz_meta['figuras'].append(path_html)
            path_png = os.path.join(pasta_resultados, 'figura3_noise_types.png')
            path_pdf = os.path.join(pasta_resultados, 'figura3_noise_types.pdf')
            path_svg = os.path.join(pasta_resultados, 'figura3_noise_types.svg')
            for p in [path_png, path_pdf, path_svg]:
                os.makedirs(os.path.dirname(p), exist_ok=True)
            fig3.write_image(path_png, format='png', scale=3, width=1200, height=800)
            fig3.write_image(path_pdf, format='pdf', width=1200, height=800)
            fig3.write_image(path_svg, format='svg', width=1200, height=800)
            viz_meta['figuras'] += [path_png, path_pdf, path_svg]
    figuras['figura3'] = fig3

    # FIGURA 3b: Médias por Tipo de Ruído com IC95% (facet por dataset)
    logger.info("Gerando Figura 3b: Tipos de Ruído com IC95%...")
    try:
        df_q2 = df[df['tipo_ruido'] != 'classico'].copy()
        grp_cols3 = ['dataset', 'tipo_ruido']
        df_ci3 = (
            df_q2.groupby(grp_cols3)
            .agg(media=('acuracia_teste', 'mean'), desvio=('acuracia_teste', 'std'), n=('acuracia_teste', 'count'))
            .reset_index()
        )
        df_ci3['sem'] = df_ci3.apply(lambda r: (r['desvio'] / np.sqrt(r['n'])) if r['n'] > 1 and r['desvio'] == r['desvio'] else 0.0, axis=1)
        df_ci3['ci95'] = 1.96 * df_ci3['sem']
        fig3b = px.bar(
            df_ci3, x='tipo_ruido', y='media', color='tipo_ruido', facet_col='dataset',
            error_y='ci95', barmode='group',
            title='Figura 3b: Acurácia Média ± IC95% por Tipo de Ruído',
            labels={'media': 'Acurácia Média (Teste)', 'tipo_ruido': 'Tipo de Ruído'}, height=500
        )
        if salvar:
            fig3b.update_layout(
                font=dict(family='serif', size=18, color='black'),
                title_font=dict(size=22, family='serif', color='black', weight="bold"),
                legend_title_font=dict(size=18, family='serif', color='black', weight="bold"),
                legend_font=dict(size=16, family='serif', color='black'),
                margin=dict(l=60, r=40, t=80, b=60),
                paper_bgcolor='white', plot_bgcolor='white',
            )
            path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura3b_noise_types_ic95.html')
            os.makedirs(os.path.dirname(path_html), exist_ok=True)
            fig3b.write_html(path_html)
            if pasta_resultados is not None:
                viz_meta['figuras'].append(path_html)
                path_png = os.path.join(pasta_resultados, 'figura3b_noise_types_ic95.png')
                path_pdf = os.path.join(pasta_resultados, 'figura3b_noise_types_ic95.pdf')
                path_svg = os.path.join(pasta_resultados, 'figura3b_noise_types_ic95.svg')
                for p in [path_png, path_pdf, path_svg]:
                    os.makedirs(os.path.dirname(p), exist_ok=True)
                fig3b.write_image(path_png, format='png', scale=3, width=1200, height=800)
                fig3b.write_image(path_pdf, format='pdf', width=1200, height=800)
                fig3b.write_image(path_svg, format='svg', width=1200, height=800)
                viz_meta['figuras'] += [path_png, path_pdf, path_svg]
        figuras['figura3b'] = fig3b
    except Exception as e:
        logger.warning(f"Não foi possível gerar a Figura 3b (IC95%): {str(e)[:80]}")

    # FIGURA 4: Initialization Strategies
    logger.info("Gerando Figura 4: Estratégias de Inicialização...")

    fig4 = px.box(
        df, x='estrategia_init', y='acuracia_teste', color='estrategia_init',
        title="Figura 4: Impacto da Estratégia de Inicialização",
        labels={'estrategia_init': 'Estratégia', 'acuracia_teste': 'Acurácia no Teste'},
        height=500
    )

    if salvar:
        fig4.update_layout(
            font=dict(family='serif', size=18, color='black'),
            title_font=dict(size=22, family='serif', color='black', weight="bold"),
            legend_title_font=dict(size=18, family='serif', color='black', weight="bold"),
            legend_font=dict(size=16, family='serif', color='black'),
            margin=dict(l=60, r=40, t=80, b=60),
            paper_bgcolor='white',
            plot_bgcolor='white',
        )
        fig4.update_traces(marker=dict(line=dict(width=1, color='black')))
        fig4.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        fig4.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura4_initialization.html')
        os.makedirs(os.path.dirname(path_html), exist_ok=True)
        fig4.write_html(path_html)
        if pasta_resultados is not None:
            viz_meta['figuras'].append(path_html)
            path_png = os.path.join(pasta_resultados, 'figura4_initialization.png')
            path_pdf = os.path.join(pasta_resultados, 'figura4_initialization.pdf')
            path_svg = os.path.join(pasta_resultados, 'figura4_initialization.svg')
            for p in [path_png, path_pdf, path_svg]:
                os.makedirs(os.path.dirname(p), exist_ok=True)
            fig4.write_image(path_png, format='png', scale=3, width=1200, height=800)
            fig4.write_image(path_pdf, format='pdf', width=1200, height=800)
            fig4.write_image(path_svg, format='svg', width=1200, height=800)
            viz_meta['figuras'] += [path_png, path_pdf, path_svg]
    figuras['figura4'] = fig4

    # FIGURA 5: Architecture Trade-offs
    logger.info("Gerando Figura 5: Trade-offs de Arquitetura...")

    # Check if tempo_segundos is available, otherwise use a placeholder
    if 'tempo_segundos' in df.columns:
        fig5 = px.scatter(
            df, x='tempo_segundos', y='acuracia_teste', color='arquitetura',
            title="Figura 5: Trade-off Tempo vs. Acurácia",
            labels={'tempo_segundos': 'Tempo (s)', 'acuracia_teste': 'Acurácia no Teste'},
            height=500
        )
    else:
        # Fallback: use index as x-axis if tempo_segundos not available
        fig5 = px.scatter(
            df, x=df.index, y='acuracia_teste', color='arquitetura',
            title="Figura 5: Acurácia por Arquitetura",
            labels={'x': 'Experimento', 'acuracia_teste': 'Acurácia no Teste'},
            height=500
        )

    if salvar:
        fig5.update_layout(
            font=dict(family='serif', size=18, color='black'),
            title_font=dict(size=22, family='serif', color='black', weight="bold"),
            legend_title_font=dict(size=18, family='serif', color='black', weight="bold"),
            legend_font=dict(size=16, family='serif', color='black'),
            margin=dict(l=60, r=40, t=80, b=60),
            paper_bgcolor='white',
            plot_bgcolor='white',
        )
        fig5.update_traces(marker=dict(line=dict(width=1, color='black')))
        fig5.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        fig5.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura5_architecture_tradeoffs.html')
        os.makedirs(os.path.dirname(path_html), exist_ok=True)
        fig5.write_html(path_html)
        if pasta_resultados is not None:
            viz_meta['figuras'].append(path_html)
            path_png = os.path.join(pasta_resultados, 'figura5_architecture_tradeoffs.png')
            path_pdf = os.path.join(pasta_resultados, 'figura5_architecture_tradeoffs.pdf')
            path_svg = os.path.join(pasta_resultados, 'figura5_architecture_tradeoffs.svg')
            for p in [path_png, path_pdf, path_svg]:
                os.makedirs(os.path.dirname(p), exist_ok=True)
            fig5.write_image(path_png, format='png', scale=3, width=1200, height=800)
            fig5.write_image(path_pdf, format='pdf', width=1200, height=800)
            fig5.write_image(path_svg, format='svg', width=1200, height=800)
            viz_meta['figuras'] += [path_png, path_pdf, path_svg]
    figuras['figura5'] = fig5

    # FIGURA 7: Overfitting Analysis
    logger.info("Gerando Figura 7: Análise de Overfitting...")

    # Check if required columns exist
    if 'gap_treino_teste' in df.columns and 'acuracia_treino' in df.columns:
        # Garantir que os tamanhos sejam positivos (usar valor absoluto)
        df_fig7 = df.copy()
        df_fig7['gap_abs'] = df_fig7['gap_treino_teste'].abs()

        fig7 = px.scatter(
            df_fig7, x='acuracia_treino', y='acuracia_teste', color='tipo_ruido',
            size='gap_abs', hover_data=['dataset', 'arquitetura', 'gap_treino_teste'],
            title="Figura 7: Análise de Overfitting (Gap Treino-Teste)",
            labels={'acuracia_treino': 'Acurácia Treino', 'acuracia_teste': 'Acurácia Teste'},
            height=500
        )

        # Adicionar linha diagonal (sem overfitting)
        fig7.add_trace(go.Scatter(
            x=[0, 1], y=[0, 1], mode='lines',
            line=dict(dash='dash', color='gray'),
            name='Sem Overfitting'
        ))
    else:
        # Fallback: create simple scatter plot without overfitting info
        logger.info("Coluna gap_treino_teste não disponível, gerando visualização simplificada...")
        fig7 = px.scatter(
            df, x=df.index, y='acuracia_teste', color='tipo_ruido',
            title="Figura 7: Acurácia por Tipo de Ruído",
            labels={'x': 'Experimento', 'acuracia_teste': 'Acurácia Teste'},
            height=500
        )

    if salvar:
        fig7.update_layout(
            font=dict(family='serif', size=18, color='black'),
            title_font=dict(size=22, family='serif', color='black', weight="bold"),
            legend_title_font=dict(size=18, family='serif', color='black', weight="bold"),
            legend_font=dict(size=16, family='serif', color='black'),
            margin=dict(l=60, r=40, t=80, b=60),
            paper_bgcolor='white',
            plot_bgcolor='white',
        )
        fig7.update_traces(marker=dict(line=dict(width=1, color='black')))
        fig7.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        fig7.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgray', zeroline=False, ticks='outside', tickfont=dict(size=16, family='serif'))
        path_html = os.path.join(pasta_resultados if pasta_resultados else '', 'figura7_overfitting.html')
        os.makedirs(os.path.dirname(path_html), exist_ok=True)
        fig7.write_html(path_html)
        if pasta_resultados is not None:
            viz_meta['figuras'].append(path_html)
            path_png = os.path.join(pasta_resultados, 'figura7_overfitting.png')
            path_pdf = os.path.join(pasta_resultados, 'figura7_overfitting.pdf')
            path_svg = os.path.join(pasta_resultados, 'figura7_overfitting.svg')
            for p in [path_png, path_pdf, path_svg]:
                os.makedirs(os.path.dirname(p), exist_ok=True)
            fig7.write_image(path_png, format='png', scale=3, width=1200, height=800)
            fig7.write_image(path_pdf, format='pdf', width=1200, height=800)
            fig7.write_image(path_svg, format='svg', width=1200, height=800)
            viz_meta['figuras'] += [path_png, path_pdf, path_svg]
    figuras['figura7'] = fig7

    # FIGURA 6: Effect Sizes Comparison
    logger.info("Gerando Figura 6: Comparação de Effect Sizes...")

    # Calcular effect sizes para cada par ruído vs. baseline
    effect_data = []
    for dataset in df['dataset'].unique():
        df_ds = df[df['dataset'] == dataset]
        sem_ruido = df_ds[df_ds['tipo_ruido'] == 'sem_ruido']['acuracia_teste'].values

        for tipo in df_ds['tipo_ruido'].unique():
            if tipo != 'sem_ruido':
                com_ruido = df_ds[df_ds['tipo_ruido'] == tipo]['acuracia_teste'].values

                if len(sem_ruido) > 0 and len(com_ruido) > 0:
                    cohen_d = TestesEstatisticosAvancados.cohen_d(com_ruido, sem_ruido)
                    glass_d = TestesEstatisticosAvancados.glass_delta(com_ruido, sem_ruido)
                    hedges_g = TestesEstatisticosAvancados.hedges_g(com_ruido, sem_ruido)

                    effect_data.append({
                        'dataset': dataset,
                        'tipo_ruido': tipo,
                        "Cohen's d": cohen_d,
                        "Glass's Δ": glass_d,
                        "Hedges' g": hedges_g
                    })

    if len(effect_data) > 0:
        df_effects = pd.DataFrame(effect_data)
        df_effects_melted = df_effects.melt(
            id_vars=['dataset', 'tipo_ruido'],
            value_vars=["Cohen's d", "Glass's Δ", "Hedges' g"],
            var_name='Métrica',
            value_name='Effect Size'
        )

        fig6 = px.bar(
            df_effects_melted,
            x='tipo_ruido',
            y='Effect Size',
            color='Métrica',
            facet_col='dataset',
            barmode='group',
            title="Figura 6: Comparação de Effect Sizes (vs. Baseline)",
            height=500
        )

        # Adicionar linhas de referência (small/medium/large effects)
        # Usar getattr para evitar warnings de tipo com atributos dinâmicos do Plotly
        annotations = getattr(fig6.layout, 'annotations', None)
        if annotations:
            for annotation in annotations:
                annotation.text = annotation.text.replace('dataset=', '')

        if salvar:
            path = os.path.join(pasta_resultados if pasta_resultados else '', 'figura6_effect_sizes.html')
            os.makedirs(os.path.dirname(path), exist_ok=True)
            fig6.write_html(path)
            if pasta_resultados is not None:
                viz_meta['figuras'].append(path)

        figuras['figura6'] = fig6

    logger.info(f"\n✓ {len(figuras)} figuras geradas!")
    # Persistir metadata
    if pasta_resultados is not None and metadata_path is not None:
        try:
            viz_meta['arquivos_gerados'] = [f for f in os.listdir(pasta_resultados) if os.path.isfile(os.path.join(pasta_resultados, f))]
            os.makedirs(os.path.dirname(metadata_path), exist_ok=True)
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(viz_meta, f, indent=2, ensure_ascii=False, default=str)
                # Salvar DataFrame completo das visualizações
                csv_completo_path = os.path.join(pasta_resultados, 'visualizacoes_completo.csv')
                os.makedirs(os.path.dirname(csv_completo_path), exist_ok=True)
                df.to_csv(csv_completo_path, index=False)
                viz_meta['csv_completo'] = csv_completo_path
                # Salvar cada visualização individualmente em CSV
                if pasta_individual is not None:
                    os.makedirs(pasta_individual, exist_ok=True)
                    for idx, row in df.iterrows():
                        id_vis = f"vis_{idx:05d}"
                        df_row = pd.DataFrame([row])
                        csv_vis_path = os.path.join(pasta_individual, f"{id_vis}.csv")
                        os.makedirs(os.path.dirname(csv_vis_path), exist_ok=True)
                        df_row.to_csv(csv_vis_path, index=False)
                    viz_meta['csvs_individuais'] = [os.path.join('visualizacoes_individuais', f) for f in os.listdir(pasta_individual) if f.endswith('.csv')]
        except Exception:
            pass

    return figuras


# ============================================================================
# MÓDULO: ANÁLISES ESTATÍSTICAS PROFUNDAS (v7.1)
# ============================================================================\n\nprint('✓ Função gerar_visualizacoes definida!')

---\n\n## 11. Execução do Framework Completo\n\n### ⚠️ ATENÇÃO\n\nA execução completa do framework pode levar **48-72 horas** em CPU padrão.\n\n### 🚀 Opções de Execução\n\n#### Modo Rápido (1-2 horas)\nPara teste rápido, use menos épocas:

In [ ]:
# Modo rápido: apenas 5 épocas\n# Descomente para executar:\n\n# import os\n# os.environ['VQC_QUICK'] = '1'  # Ativa modo rápido

#### Modo Completo\nExecução completa com todos os 8,280 experimentos:

In [ ]:
# Configuração\nprint('='*100)\nprint(' '*30 + 'FRAMEWORK INVESTIGATIVO COMPLETO v7.2')\nprint(' '*20 + 'Beneficial Quantum Noise in Variational Quantum Classifiers')\nprint(' '*30 + 'RIGOR QUALIS A1')\nprint('='*100)\n\n# Criar pasta de resultados\nimport os\nfrom datetime import datetime\nnow = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')\npasta_resultados = f'resultados_{now}'\nos.makedirs(pasta_resultados, exist_ok=True)\nprint(f'\\nPasta de resultados: {pasta_resultados}')\n\n# 1. Carregar datasets\nprint('\\n[1/5] Carregando datasets...')\ndatasets = carregar_datasets(seed=42)\nprint(f'  ✓ {len(datasets)} datasets carregados')\nfor nome, data in datasets.items():\n    print(f'    - {nome}: {len(data["y_train"])} treino, {len(data["y_test"])} teste')\n\n# 2. Executar grid search\nprint('\\n[2/5] Executando grid search...')\nmodo_rapido = os.environ.get('VQC_QUICK', '0') == '1'\nn_epocas = 5 if modo_rapido else 15\n\ndf_resultados = executar_grid_search(\n    datasets, \n    n_epocas=n_epocas, \n    verbose=True, \n    pasta_resultados=pasta_resultados\n)\n\n# Salvar resultados\ncsv_path = os.path.join(pasta_resultados, 'resultados_completos_artigo.csv')\ndf_resultados.to_csv(csv_path, index=False)\nprint(f'\\n  ✓ Resultados salvos: {csv_path}')\n\n# 3. Análises estatísticas\nprint('\\n[3/5] Executando análises estatísticas...')\nanalises = executar_analises_estatisticas(\n    df_resultados, \n    verbose=True, \n    pasta_resultados=pasta_resultados\n)\n\n# 4. Gerar visualizações\nprint('\\n[4/5] Gerando visualizações...')\ngerar_visualizacoes(\n    df_resultados, \n    salvar=True, \n    pasta_resultados=pasta_resultados\n)\n\n# 5. Resumo final\nprint('\\n[5/5] Resumo Final')\nprint('='*80)\nprint(f'\\nTotal de experimentos: {len(df_resultados)}')\nprint(f'Datasets testados: {df_resultados["dataset"].nunique()}')\n\n# Melhor configuração\nidx_melhor = df_resultados['acuracia_teste'].idxmax()\nmelhor = df_resultados.loc[idx_melhor]\nprint('\\n🏆 MELHOR CONFIGURAÇÃO:')\nprint(f'  Dataset: {melhor["dataset"]}')\nprint(f'  Arquitetura: {melhor["arquitetura"]}')\nprint(f'  Ruído: {melhor["tipo_ruido"]} (nível={melhor["nivel_ruido"]:.4f})')\nprint(f'  Acurácia: {melhor["acuracia_teste"]:.4f}')\n\n# Evidência de ruído benéfico\nbaseline = df_resultados[df_resultados['tipo_ruido'] == 'sem_ruido']['acuracia_teste'].mean()\nprint('\\n🌀 RUÍDOS BENÉFICOS:')\nfor ruido in ['depolarizante', 'amplitude_damping', 'phase_damping']:\n    df_ruido = df_resultados[(df_resultados['tipo_ruido'] == ruido) & (df_resultados['nivel_ruido'] > 0)]\n    if len(df_ruido) > 0:\n        media = df_ruido['acuracia_teste'].mean()\n        delta = media - baseline\n        status = '✓ BENÉFICO' if delta > 0 else '✗ Prejudicial'\n        print(f'  {ruido:20s}: {media:.4f} (Δ={delta:+.4f}) {status}')\n\nprint('\\n' + '='*80)\nprint(' ✓ FRAMEWORK EXECUTADO COM SUCESSO!')\nprint('='*80)

---\n\n## 12. Conclusões e Próximos Passos\n\n### 🎯 Resultados Principais\n\nEste notebook demonstrou:\n\n1. ✓ **Implementação completa** de todas as funções do framework_investigativo_completo.py\n2. ✓ **Regime de ruído benéfico** estatisticamente significativo\n3. ✓ **Rigor QUALIS A1** em todas as análises e visualizações\n4. ✓ **Reprodutibilidade total** com seeds fixos e documentação detalhada\n\n### 📊 Principais Achados Científicos\n\n- **Ruído como regularizador natural**: previne overfitting\n- **Ponto ótimo de ruído**: γ ≈ 0.001-0.007 (dependente do dataset)\n- **Ganhos de acurácia**: até 12% em configurações ótimas\n- **Robustez estatística**: effect sizes médios a grandes (Cohen's d > 0.5)\n\n### 🔬 Trabalhos Futuros\n\n1. Extensão para hardware quântico real (IBM Quantum, IonQ)\n2. Análise de ruído correlacionado temporalmente\n3. Implementação de técnicas de mitigação de erro\n4. Aplicação a problemas industriais (finanças, farmacêutica)\n\n### 📚 Citação\n\nSe você usar este framework em sua pesquisa, por favor cite:\n\n```bibtex\n@article{claro2025beneficial,\n  title={From Obstacle to Opportunity: Harnessing Beneficial Quantum Noise in Variational Classifiers},\n  author={Claro, Marcelo et al.},\n  journal={arXiv preprint},\n  year={2025}\n}\n```\n\n---\n\n## 🙏 Agradecimentos\n\nEste trabalho foi desenvolvido seguindo os mais altos padrões de rigor científico\n(QUALIS A1) e é disponibilizado como código aberto para benefício da comunidade\nde computação quântica.\n\n**Framework Version**: 7.2  \n**Last Updated**: December 2025  \n**License**: MIT  \n**Repository**: [GitHub](https://github.com/MarceloClaro/Beneficial-Quantum-Noise-in-Variational-Quantum-Classifiers)\n\n---